# Deep Research Sustainability Screening Agent
This workdlow uses a hybrid Parallelization and Evaluator–Optimizer workflow pattern to perform deep research on a company’s adoption of innovative sustainable technologies. It enables investment teams to assess ESG leadership using dynamic, internet-scale data.

## Workflow Structure
![sustainability screening workflow](../assets/svgs/sustainability_screening.drawio.svg)


The following code was adapted from [Google Gemini's Langgraph Research Agent Repo](https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart) and Langgraphs workflow logic. For more details on Langgraph workflows check out the guides [here](https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining).

## Install Dependencies


In [ ]:
pip install langgraph langchain-google-genai langchain-openai pydantic python-dotenv langchain-core google-genai 

## Configuration
First we need to define the models and parameters for each step in the workflow

In [1]:
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableConfig
from typing import Any, Optional
import os

class Configuration(BaseModel):
    """The configuration for the agent."""

    # Must be a google model
    query_generator_model: str = Field(
        default="gemini-2.0-flash",
        metadata={
            "description": "The name of the language model to use for the agent's query generation."
        },
    )

    reflection_model: str = Field(
        default="gpt-5-mini-2025-08-07",
        metadata={
            "description": "The name of the language model to use for the agent's reflection."
        },
    )

    reasoning_model: str = Field(
        default='gpt-5-mini-2025-08-07',
        metadata={"description": "The reasoning model for complex analysis."},
    )

    number_of_initial_queries: int = Field(
        default=3,
        metadata={"description": "The number of initial search queries to generate."},
    )

    max_research_loops: int = Field(
        default=2,
        metadata={"description": "The maximum number of research loops to perform."},
    )
    
    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )

        # Get raw values from environment or config
        raw_values: dict[str, Any] = {
            name: os.environ.get(name.upper(), configurable.get(name))
            for name in cls.model_fields.keys()
        }

        # Filter out None values
        values = {k: v for k, v in raw_values.items() if v is not None}

        return cls(**values)


## State
Here we add memory to so variables can presist across nodes in the workflow.

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator

class OverallState(TypedDict):
    messages: Annotated[list, add_messages]
    sources_gathered: Annotated[list, operator.add]
    sources_used:Annotated[list, operator.add]
    web_research_result: Annotated[list, operator.add]
    search_query: Annotated[list, operator.add]
    initial_search_query_count: int
    max_research_loops: int
    research_loop_count: int
    

class Query(TypedDict):
    query: str
    rationale: str

class QueryGenerationState(TypedDict):
    query_list: list[Query]

class WebSearchState(TypedDict):
    search_query: str
    id: str

class ReflectionState(TypedDict):
    is_sufficient: bool
    knowledge_gap: str
    follow_up_queries: Annotated[list, operator.add]
    research_loop_count: int
    number_of_ran_queries: int

## Utils
These are helper functions to extract key variables at various steps in the workflow

In [3]:
from typing import Any, Dict, List
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage
from datetime import datetime

def get_current_date():
    return datetime.now().strftime("%B %d, %Y")

def get_research_topic(messages: List[AnyMessage]) -> str:
    """
    Get the research topic from the messages.
    """
    # check if request has a history and combine the messages into a single string
    if len(messages) == 1:
        research_topic = messages[-1].content
    else:
        research_topic = ""
        for message in messages:
            if isinstance(message, HumanMessage):
                research_topic += f"User: {message.content}\n"
            elif isinstance(message, AIMessage):
                research_topic += f"Assistant: {message.content}\n"
    return research_topic


def resolve_urls(urls_to_resolve: List[Any], id: int) -> Dict[str, str]:
    """
    Create a map of the vertex ai search urls (very long) to a short url with a unique id for each url.
    Ensures each original URL gets a consistent shortened form while maintaining uniqueness.
    """
    prefix = f"https://vertexaisearch.cloud.google.com/id/"
    urls = [site.web.uri for site in urls_to_resolve]

    # Create a dictionary that maps each unique URL to its first occurrence index
    resolved_map = {}
    for idx, url in enumerate(urls):
        if url not in resolved_map:
            resolved_map[url] = f"{prefix}{id}-{idx}"

    return resolved_map


def insert_citation_markers(text, citations_list):
    """
    Inserts citation markers into a text string based on start and end indices.

    Args:
        text (str): The original text string.
        citations_list (list): A list of dictionaries, where each dictionary
                               contains 'start_index', 'end_index', and
                               'segment_string' (the marker to insert).
                               Indices are assumed to be for the original text.

    Returns:
        str: The text with citation markers inserted.
    """
    # Sort citations by end_index in descending order.
    # If end_index is the same, secondary sort by start_index descending.
    # This ensures that insertions at the end of the string don't affect
    # the indices of earlier parts of the string that still need to be processed.
    sorted_citations = sorted(
        citations_list, key=lambda c: (c["end_index"], c["start_index"]), reverse=True
    )

    modified_text = text
    for citation_info in sorted_citations:
        # These indices refer to positions in the *original* text,
        # but since we iterate from the end, they remain valid for insertion
        # relative to the parts of the string already processed.
        end_idx = citation_info["end_index"]
        marker_to_insert = ""
        for segment in citation_info["segments"]:
            marker_to_insert += f" [{segment['label']}]({segment['short_url']})"
        # Insert the citation marker at the original end_idx position
        modified_text = (
            modified_text[:end_idx] + marker_to_insert + modified_text[end_idx:]
        )

    return modified_text


def get_citations(response, resolved_urls_map):
    """
    Extracts and formats citation information from a Gemini model's response.

    This function processes the grounding metadata provided in the response to
    construct a list of citation objects. Each citation object includes the
    start and end indices of the text segment it refers to, and a string
    containing formatted markdown links to the supporting web chunks.

    Args:
        response: The response object from the Gemini model, expected to have
                  a structure including `candidates[0].grounding_metadata`.
                  It also relies on a `resolved_map` being available in its
                  scope to map chunk URIs to resolved URLs.

    Returns:
        list: A list of dictionaries, where each dictionary represents a citation
              and has the following keys:
              - "start_index" (int): The starting character index of the cited
                                     segment in the original text. Defaults to 0
                                     if not specified.
              - "end_index" (int): The character index immediately after the
                                   end of the cited segment (exclusive).
              - "segments" (list[str]): A list of individual markdown-formatted
                                        links for each grounding chunk.
              - "segment_string" (str): A concatenated string of all markdown-
                                        formatted links for the citation.
              Returns an empty list if no valid candidates or grounding supports
              are found, or if essential data is missing.
    """
    citations = []

    # Ensure response and necessary nested structures are present
    if not response or not response.candidates:
        return citations

    candidate = response.candidates[0]
    if (
        not hasattr(candidate, "grounding_metadata")
        or not candidate.grounding_metadata
        or not hasattr(candidate.grounding_metadata, "grounding_supports")
    ):
        return citations

    for support in candidate.grounding_metadata.grounding_supports:
        citation = {}

        # Ensure segment information is present
        if not hasattr(support, "segment") or support.segment is None:
            continue  # Skip this support if segment info is missing

        start_index = (
            support.segment.start_index
            if support.segment.start_index is not None
            else 0
        )

        # Ensure end_index is present to form a valid segment
        if support.segment.end_index is None:
            continue  # Skip if end_index is missing, as it's crucial

        # Add 1 to end_index to make it an exclusive end for slicing/range purposes
        # (assuming the API provides an inclusive end_index)
        citation["start_index"] = start_index
        citation["end_index"] = support.segment.end_index

        citation["segments"] = []
        if (
            hasattr(support, "grounding_chunk_indices")
            and support.grounding_chunk_indices
        ):
            for ind in support.grounding_chunk_indices:
                try:
                    chunk = candidate.grounding_metadata.grounding_chunks[ind]
                    resolved_url = resolved_urls_map.get(chunk.web.uri, None)
                    citation["segments"].append(
                        {
                            "label": chunk.web.title.split(".")[:-1][0],
                            "short_url": resolved_url,
                            "value": chunk.web.uri,
                        }
                    )
                except (IndexError, AttributeError, NameError):
                    # Handle cases where chunk, web, uri, or resolved_map might be problematic
                    # For simplicity, we'll just skip adding this particular segment link
                    # In a production system, you might want to log this.
                    pass
        citations.append(citation)
    return citations


## Prompts
Here we define the prompts for the different nodes in the workflow.

In [4]:
# ==========================================
# 1. RESEARCH AGENT PROMPTS
# ==========================================
query_writer_instructions = """Your goal is to generate sophisticated and diverse web search queries. These queries are intended for an advanced automated web research tool capable of analyzing complex results, following links, and synthesizing information.

Instructions:
- Always prefer a single search query, only add another query if the original question requests multiple aspects or elements and one query is not enough.
- Each query should focus on one specific aspect of the original question.
- Don't produce more than {number_queries} queries.
- Queries should be diverse, if the topic is broad, generate more than 1 query.
- Don't generate multiple similar queries, 1 is enough.
- Query should ensure that the most current information is gathered. The current date is {current_date}.

Format: 
- Format your response as a JSON object with ALL three of these exact keys:
   - "rationale": Brief explanation of why these queries are relevant
   - "query": A list of search queries

Example:

Topic: What revenue grew more last year apple stock or the number of people buying an iphone
```json
{{
    "rationale": "To answer this comparative growth question accurately, we need specific data points on Apple's stock performance and iPhone sales metrics. These queries target the precise financial information needed: company revenue trends, product-specific unit sales figures, and stock price movement over the same fiscal period for direct comparison.",
    "query": ["Apple total revenue growth fiscal year 2024", "iPhone unit sales growth fiscal year 2024", "Apple stock price growth fiscal year 2024"],
}}
```

Context: {research_topic}"""

web_searcher_instructions = """Conduct targeted Google Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is {current_date}.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
{research_topic}
"""

reflection_instructions = """You are an expert research assistant analyzing summaries about "{research_topic}".

Instructions:
- Identify knowledge gaps or areas that need deeper exploration and generate a follow-up query. (1 or multiple).
- If provided summaries are sufficient to answer the user's question, don't generate a follow-up query.
- If there is a knowledge gap, generate a follow-up query that would help expand your understanding.
- Focus on technical details, implementation specifics, or emerging trends that weren't fully covered.

Requirements:
- Ensure the follow-up query is self-contained and includes necessary context for web search.

Output Format:
- Format your response as a JSON object with these exact keys:
   - "is_sufficient": true or false
   - "knowledge_gap": Describe what information is missing or needs clarification
   - "follow_up_queries": Write a specific question to address this gap

Example:
```json
{{
    "is_sufficient": true, // or false
    "knowledge_gap": "The summary lacks information about performance metrics and benchmarks", // "" if is_sufficient is true
    "follow_up_queries": ["What are typical performance benchmarks and metrics used to evaluate [specific technology]?"] // [] if is_sufficient is true
}}
```

Reflect carefully on the Summaries to identify knowledge gaps and produce a follow-up query. Then, produce your output following this JSON format:

Summaries:
{summaries}
"""

answer_instructions = """Generate a high-quality answer to the user's question based on the provided summaries.

Instructions:
- The current date is {current_date}.
- You are the final step of a multi-step research process, don't mention that you are the final step. 
- You have access to all the information gathered from the previous steps.
- You have access to the user's question.
- Generate a high-quality answer to the user's question based on the provided summaries and the user's question.
- You MUST include all the citations from the summaries in the answer correctly.
- You MUST format your answer in markdown and include links formatted as [label](link)!
- Do not provide number labels. Use the proper link label.

User Question:
"{users_question}"

User Context:
- {research_topic}

Summaries:
{summaries}"""



## Schemas
These are the data models used to produce structured outputs from the LLMs.

In [5]:
from pydantic import BaseModel, Field
from typing import List

class SearchQueryList(BaseModel):
    query: List[str] = Field(
        description="A list of search queries to be used for web research."
    )
    rationale: str = Field(
        description="A brief explanation of why these queries are relevant to the research topic."
    )

class Reflection(BaseModel):
    is_sufficient: bool = Field(
        description="Whether the provided summaries are sufficient to answer the user's question."
    )
    knowledge_gap: str = Field(
        description="A description of what information is missing or needs clarification."
    )
    follow_up_queries: List[str] = Field(
        description="A list of follow-up queries to address the knowledge gap."
    )

## Nodes
Here we define each node of the workflow. 

In [6]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from google.genai import Client
from langgraph.types import Send

from langchain_core.messages import AIMessage
from logging_decorator import log_node_execution, log_node

from dotenv import load_dotenv

load_dotenv()

genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))

@log_node_execution("🔍 Query Generation")
def generate_queries(state: OverallState, config: RunnableConfig) -> QueryGenerationState:
    """LangGraph node that generates search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    # Get configurable values
    configurable = Configuration.from_runnable_config(config)
    model = configurable.query_generator_model
    
    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries or 3

    if 'gemini' in model.lower():
        llm = ChatGoogleGenerativeAI(
            model=model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in model.lower()) | ('o3' in model.lower()) | ('o4' in model.lower()) | ('o1' in model.lower()):
        llm = ChatOpenAI(
            model=model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )

    structured_llm = llm.with_structured_output(SearchQueryList)

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    return {"query_list": result.query}

@log_node_execution("🌐 Web Research")
def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configuration for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Get configurable values
    configurable = Configuration.from_runnable_config(config)
    
    # Configure
    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=state["search_query"],
    )
    
    # Get model name and ensure it works with Google Search
    model = configurable.query_generator_model or 'gemini-2.0-flash'

    if 'gemini' not in model:
        print("Model not supported for search. Defaulting to google model.")
        model = 'gemini-2.0-flash'

    # Uses the google genai client as the langchain client doesn't return grounding metadata
    response = genai_client.models.generate_content(
        model=model,
        contents=formatted_prompt,
        config={
            "tools": [{"google_search": {}}],
            "temperature": 0,
        },
    )
    # resolve the urls to short urls for saving tokens and time
    resolved_urls = resolve_urls(
        response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
    )

    # Gets the citations and adds them to the generated text
    citations = get_citations(response,resolved_urls)
    modified_text = insert_citation_markers(response.text, citations)
    sources_gathered = [item for citation in citations for item in citation["segments"]]

    return {
        "sources_gathered": sources_gathered,
        "search_query": [state["search_query"]],
        "web_research_result": [modified_text],
    }

@log_node_execution("🤔 Reflection")
def reflection(state: OverallState, config: RunnableConfig) -> ReflectionState:
    """LangGraph node that identifies knowledge gaps and generates potential follow-up queries.

    Analyzes the current summary to identify areas for further research and generates
    potential follow-up queries. Uses structured output to extract
    the follow-up query in JSON format.

    Args:
        state: Current graph state containing the running summary and research topic
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated follow-up query
    """
    configurable = Configuration.from_runnable_config(config)
    # Increment the research loop count and get the reasoning model
    state["research_loop_count"] = state.get("research_loop_count", 0) + 1
    reasoning_model = configurable.reasoning_model or {}

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = reflection_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n\n---\n\n".join(state["web_research_result"]),
    )
    if 'gemini' in reasoning_model.lower():
        llm = ChatGoogleGenerativeAI(
            model=reasoning_model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in reasoning_model.lower()) | ('o3' in reasoning_model.lower()) | ('o4' in reasoning_model.lower()) | ('o1' in reasoning_model.lower()):
        llm = ChatOpenAI(
            model=reasoning_model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )
        
    result = llm.with_structured_output(Reflection).invoke(formatted_prompt)

    return {
        "is_sufficient": result.is_sufficient,
        "knowledge_gap": result.knowledge_gap,
        "follow_up_queries": result.follow_up_queries,
        "research_loop_count": state["research_loop_count"],
        "number_of_ran_queries": len(state["search_query"]),
    }

@log_node_execution("📋 Finalize Answer")
def finalize_answer(state: OverallState, config: RunnableConfig) -> OverallState:
    """LangGraph node that finalizes the research summary.

    Prepares the final output by combining research results into a well-structured
    research report with proper citations and working markdown links.

    Args:
        state: Current graph state containing the running summary and sources gathered

    Returns:
        Dictionary with state update, including the final answer with working citations
    """
    configurable = Configuration.from_runnable_config(config)
    reasoning_model = configurable.reasoning_model or {}

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = answer_instructions.format(
        current_date=current_date,
        users_question=state['messages'][0],
        research_topic=get_research_topic(state["messages"]),
        summaries="\n---\n\n".join(state["web_research_result"]),
    )

    if 'gemini' in reasoning_model.lower():
        llm = ChatGoogleGenerativeAI(
            model=reasoning_model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in reasoning_model.lower()) | ('o3' in reasoning_model.lower()) | ('o4' in reasoning_model.lower()) | ('o1' in reasoning_model.lower()):
        llm = ChatOpenAI(
            model=reasoning_model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )
    else:
        "Model not supported. Only use Google or OpenAI models."

    result = llm.invoke(formatted_prompt)

    # Collect unique sources that are actually used in the final content
    unique_sources = []
    for source in state["sources_gathered"]:
        if source["short_url"] in result.content:
            result.content = result.content.replace(
                f'({source["short_url"]})', f'({source["value"]})'
            )
            unique_sources.append(source)

    return {
        "messages": [AIMessage(content=result.content)],
        "sources_used": unique_sources,
    }

# ==========================================
# CONDITIONAL EDGES
# ==========================================

@log_node
def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]

@log_node
def evaluate_research(
    state: ReflectionState,
    config: RunnableConfig,
) -> OverallState:
    """LangGraph routing function that determines the next step in the research flow.

    Controls the research loop by deciding whether to continue gathering information
    or to finalize the summary based on the configured maximum number of research loops.

    Args:
        state: Current graph state containing the research loop count
        config: Configuration for the runnable, including max_research_loops setting

    Returns:
        String literal indicating the next node to visit ("web_research" or "finalize_summary")
    """
    configurable = Configuration.from_runnable_config(config)
    max_research_loops = (
        state.get("max_research_loops")
        if state.get("max_research_loops") is not None
        else configurable.max_research_loops
    )
    if state["is_sufficient"] or state["research_loop_count"] >= max_research_loops:
        return "finalize_answer"
    else:
        return [
            Send(
                "web_research",
                {
                    "search_query": follow_up_query,
                    "id": state["number_of_ran_queries"] + int(idx),
                },
            )
            for idx, follow_up_query in enumerate(state["follow_up_queries"])
        ]

## Graph
Now we build the graph which pulls it all together.

In [7]:
from langgraph.graph import StateGraph, START, END
from state import OverallState

graph_builder = StateGraph(OverallState, config_schema=Configuration)

# ==========================================
# Define Nodes
# ==========================================

graph_builder.add_node("generate_queries",generate_queries)
graph_builder.add_node("web_research",web_research)
graph_builder.add_node("reflection",reflection)
graph_builder.add_node("finalize_answer",finalize_answer)


# ==========================================
# Define Edges
# ==========================================

graph_builder.add_edge(START, "generate_queries")  
graph_builder.add_conditional_edges(
    "generate_queries", continue_to_web_research, ["web_research"]
)
graph_builder.add_edge('web_research','reflection')
graph_builder.add_conditional_edges(
    "reflection", evaluate_research, ["web_research", "finalize_answer"]
)
graph_builder.add_edge("web_research", END)

# ==========================================
# Compile The Graph
# ==========================================

graph = graph_builder.compile()

/var/folders/3g/79hl_7s12859plnmqc8l6nrw0000gp/T/ipykernel_51657/2112391383.py:4: LangGraphDeprecatedSinceV10: `config_schema` is deprecated and will be removed. Please use `context_schema` instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph_builder = StateGraph(OverallState, config_schema=Configuration)


# Test Sustainability Workflow
Here we provide our sustainability screening prompt.

In [8]:
prompt = """Research {company_name}'s adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:

## RESEARCH AREAS:

1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainable materials) has {company_name} implemented?

2. **Innovation Leadership (25 points)**: What cutting-edge sustainable technology initiatives, R&D projects, or partnerships is {company_name} pursuing?

3. **Measurable Impact (25 points)**: What quantifiable environmental and business impacts have resulted from their sustainable technology investments?

4. **Future Strategy (15 points)**: What are {company_name}'s documented commitments and roadmap for future sustainable technology adoption?

5. **Industry Position (10 points)**: How does {company_name} compare to competitors in sustainable technology leadership and adoption?

## SCORING CRITERIA (Total: 100 points):

**Technology Adoption (25 points):**
- 20-25: Multiple breakthrough technologies implemented at scale across operations
- 15-19: Several innovative technologies deployed with significant operational integration
- 10-14: Some sustainable technologies adopted but limited scope or innovation
- 5-9: Basic sustainable technology implementations, mostly conventional solutions
- 0-4: Minimal or no significant sustainable technology adoption

**Innovation Leadership (25 points):**
- 20-25: Leading industry with proprietary sustainable tech development, major R&D investments, breakthrough patents
- 15-19: Strong innovation program with partnerships, pilot projects, and emerging tech investments
- 10-14: Moderate innovation efforts, some R&D or partnerships in sustainable tech
- 5-9: Limited innovation activities, following industry trends rather than leading
- 0-4: No significant sustainable technology innovation or R&D

**Measurable Impact (25 points):**
- 20-25: Documented major environmental improvements (>50% emissions reduction, significant resource savings)
- 15-19: Strong measurable impact (25-50% improvements in key metrics)
- 10-14: Moderate documented impact (10-25% improvements)
- 5-9: Some measurable benefits but limited scope or impact
- 0-4: No clear quantifiable environmental or efficiency benefits

**Future Strategy (15 points):**
- 13-15: Comprehensive roadmap with specific targets, timelines, and major investments committed
- 10-12: Clear strategy with some specific commitments and targets
- 7-9: General sustainability commitments with limited specific tech focus
- 4-6: Vague future plans, minimal concrete commitments
- 0-3: No clear future sustainable technology strategy

**Industry Position (10 points):**
- 9-10: Industry leader, setting standards, widely recognized for sustainable tech leadership
- 7-8: Among top performers in industry sector
- 5-6: Average compared to industry peers
- 3-4: Below average, following rather than leading
- 0-2: Laggard in industry for sustainable technology adoption

## OUTPUT REQUIREMENTS:
- Provide detailed findings for each research area
- Give specific scores for each category with justification
- Calculate and present final overall score (0-100)
- Include recent examples (2022-2025), specific metrics, and credible sources
- Focus on innovation and leadership rather than basic compliance 

## FINAL ASSESSMENT FORMAT:
**SUSTAINABLE TECHNOLOGY ADOPTION SCORE: [X]/100**

Technology Adoption: [X]/25
Innovation Leadership: [X]/25  
Measurable Impact: [X]/25
Future Strategy: [X]/15
Industry Position: [X]/10

**Overall Assessment**: [Brief summary of company's sustainable tech position]"""

### Format The Prompt With the Company of Concern

In [ ]:
company = '3M'
formated_prompt = prompt.format(company_name=company)
formated_prompt

### Run the Workflow

In [ ]:
config = Configuration

state = graph.invoke(
        {"messages": [{"role": "user", "content": formated_prompt}]}
    )
report = state["messages"][-1].content

In [12]:
from IPython.display import Markdown, display

# This will render markdown properly
display(Markdown(report))

**SUSTAINABLE TECHNOLOGY ADOPTION SCORE: 82/100**

Technology Adoption: 20/25  
Innovation Leadership: 18/25  
Measurable Impact: 22/25  
Future Strategy: 14/15  
Industry Position: 8/10

Overall Assessment: 3M is a strong performer on sustainable technologies — it has deployed multiple large-scale renewable-energy and energy-efficiency programs, installed advanced water‑treatment and circular‑materials initiatives, and moved into emerging climate technologies (e.g., materials for DAC filters). Measurable performance to 2024 shows large GHG and water-efficiency gains and big reductions in virgin plastic use. 3M scores highly for adoption and impact, slightly below the very top industry leaders on pure innovation leadership (because large-scale commercial carbon‑removal or CCUS deployments are still emerging and some technical details remain limited publicly). The company’s SBTi-validated targets, PFAS phase‑out commitment, and multi‑year investment plan further support a high overall score.

Findings and justification (by research area)

Technology Adoption (20/25) — What technologies has 3M implemented?
- Renewable electricity: 3M surpassed its 2025 renewable‑electricity goal early, reporting >50% (reported ~51.9% in 2022 and ~56.2% in 2023) of operations powered by renewables and membership in RE100. 3M also executed large PPAs (example: Gunsight wind farm PPA referenced) to secure renewable supply. [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez) [stocktitan](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHynEGjXXyd73GTl1TeynO1K0JBLYwhw9TGtZDMS__AzswoMBao9YH-JtZwTNWOFdpqR6KWBqQOW5BUh2UQ1-bf_mIly8rPJ7OcR0rNl7vpdSQsHUVlkp8aGZH2NFbJ7d8pjWW180tbdnFjzMruO4PJQUI2ChqhP57Lgmy2EPuSpYBXrcKyiPPUAaK1Faah-ldU8P4S1TeipYxKw_3v) [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQElWOquPk2yFaN5miRnh8karS5IYgVhWzh_0-xGiiKE3t8nfPR_Sl4CNLo8_KmKcCmBMOo1kGGNSemi8st29e2te7gb-E8AmGZtTZtq1-2k8BA8wym7lnL-IldYVLge7RfP0vStNoVKJBC49G0=) [rmi](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE4LGdy9AQM24Z0ZgJcTijxY2jMgYkGvCBlcrB5Q_ZXVnFX8YqhxLekOjIlAI6XJAsT3qP32LB9JxK6PCw_q4_Tq3CNVYTHqlZQHV2snJtRxQbjKoRm7PQPptWD8NfGWUpNspkbigblJXVkE3Ssiq-33lVJCOnJ2tzlSyBZWz9CHcWcGcwSKWqKiUDcfuGS) [sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG-Wr2RM_8NdQBk2BbiHkKz-EfxnU6xPRDhfc_6CQiKVYZ05C25EDWV7a26qjzuC_Od0PAEOj6r8sEHywyhhg3qa3rABEdeXdn8L6Ra0fYgb32rdn_BpfFMmEFgJ3XsFMyEc0oYXRx6LTMdd5dc83W4Ud4PNG7mbI6VjFc0u01E7-CHL65ySpmni2DjrFDzLnZrErMZLY_X87OR7f8WihiYdojnlC0tvQ==)
- Energy efficiency: ISO 50001 certification at dozens of sites, corporate programs to improve energy efficiency (target: 30% improvement indexed to net sales by 2025), and plant‑level motor/system upgrades with quantified MWh savings examples. [energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHENHk3DMsjey2XBvNkQvKUL95V6upn1IP6-ct4duvuzqXPFOH8Im9_fEUpz-fQ5mYShHWUhy5s5s6PqM8_ufbGs68WdWr3bN_uHNQvOQrTDc7VmmFu60JH6S4JJ9LZPCg4LnaB03OhlsgoU3gLVgBvczOka8fIEcq3AWADA1f8SZUzR17fs9QzOlwJE4Fxk14oNq6PZTmKVbPK) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbwHA5WshV5MMkvKM0VIPRkhcAq26xVMvjc2G5bu-6_kuNdPz2Uj2U-JwbHXt5964Y-LoSPLqJkbWMuQhF36HY7g7JLsF0T0PmaX5p6_IoH2uXofK5SIA7o75X9EJ7w6S3qj28S__NrTCii-u7BzMwVh0GtDaQ65jg_P4=) [thecsrjournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpPLE4IjUWq3i_PbMuXOwKuzoatb8l-iGlwjJ4fMdlfbe4Ke_1-ji9R0T79H5k8sgfBqk1Db3lJB4_NyENVvPNg3dvXmzSmk2t9grtmzpCsdDSbDpSmNv0KGVxCpu-GPOpM_67XiCkusAH6NlLfzjeqKgqVIy32CNCidbB)
- Water management and treatment technologies: large capital deployment of state‑of‑the‑art water purification at largest water‑using sites (granular activated carbon, reverse osmosis, ion exchange, thermal oxidation) and commitments to reduce water use 25% by 2030. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=) [wwdmag](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG60IXHB6Z7dcE8Y3gyf2qoICa8P_d2FbCaRCyP4K1fVthXP_xS7f5wsK0orGrHPp5J_Watq4yfAnF5P0sy3IVqEMkMzhQsPvvf_WZ1V1ZqRizBAzYNOiWccbsG9gn-Xfhyvu5YXUb0n9tR9U5mkxNi-eKNKS50VjXT9S1omNJuYlXQldycGG9MsDX0zfiMOfcrP5edEjCdrmGa3IkXyYH3bNVuLI4=) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF7MSaZbOPojs9yCb1tDK1UM2mdQ0ncQg5usaTYj4_reiHIscXxatCQcBPZ08eWEUIUcDQysMppb0BsWgm3XNYNlc9ryPtVWWHvAySTr4m0zs7R613Zi9XWB4jR7efqyoeBTZjH1WAwYkM_hJQy42dEa4WoB2SieMXJT-XcJZKphi6arEmtOaPpBoae-vwEdLV0MwtB9AJEE6TjH49ZVYgJjHKmTSBtTv1PPxUjpPTB)
- Circular / sustainable materials: major reductions in use of virgin fossil‑based plastics and recycled/bio content integration into products and packaging; corporate requirement that every new product deliver a Sustainability Value Commitment (SVC). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGc01DlTvg_ggSnpJS8UvHFl576Nr6FTO_HCB6ld2cNyaowk1dGivbGR2dhWIGWtFBb1kd4gQj_vH7gwXEOumqYDuWOw6Y8C8nCD3N1miy-mDhBg1C24bk_dthfGmcUivmRryMvMaN8Qzsv3bKHnWRtUSA6K3XH7t2D_Ycw6WGc6THA1mhDtfjgK9PvMLT2xMixopgieAkruxin) [issa](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpuVuwFjQd0I0aTqP4nRlbUJcbPiEnQF-DuyTjzzIKj-ZuFtcDQuGEl9sTrWNbVmtKWGZLpGG8vxA2IRsEfDKtCGj44SCXbaVK1knr9DeSUHah7-BeRae8av9USui-Y6zEeLoZiuLdxt_OoYoCWInOWYUT8QNoFczbZFA46pM1iBmEjQ==)
- Carbon‑management enabling technologies: while the company reports collaboration with customers and product development for carbon capture (e.g., scaling SOAR material for Svante DAC filters), large on‑site CCUS deployments are not yet documented at scale in public filings. [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)

Justification for score: 3M has implemented multiple advanced technologies at scale across operations (renewables, ISO‑certified efficiency programs, advanced water treatment, circular materials programs). Because some breakthrough carbon‑removal technologies are in partnership/pilot/commercial scale‑up (not yet broad on‑site CCUS), I rate adoption strong but not perfect. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez)

Innovation Leadership (18/25) — R&D, partnerships, pilot initiatives
- R&D structures and priorities: 3M aligns R&D to “Science for Circular, Science for Climate, Science for Community,” requires SVC for new products, and maintains corporate R&D and sustainability governance. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF136cZEnTUa6__Qmf3XEVrXxvQLhyfzLkkYpAzlf6-fKeCdfofGMICazGr3KQLfXfWVL0QNwCKvsUmEDIY2ewErweXbOTEcjdkTKiqjX2BhVTmkyJsk4omifdcEkGhvinigsxXWMymcA==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyHHuq3MoZ-a2DCV24gM18BExneS4U6-ykf1fbJbA7wZsDSAjyN-fXFtZLaTJFWr-Ptpj-jHZ-vwra6266bKhHnKXI3cfNrzlSRchkHNlje9fZzPjWgSeM_lq1NpEXb84W3J2tEEm7jjs0TMzYbFti24spWOi19M8y7O4B26ltm9oGRKtrRvw=) [eriks](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9ZxRxkU8-VMdJWQse6ljW-t55wv9xyAAlUOoVCSAdBDWa6CT3oBWpdIbQyL9Kvlin4poyX2JINmJ8dRHc2bN19tdISGSxYkZWhRFCVDqsiIXeS08uAjxZH3mNSFrbmJP51brYjNOhXB8L00LuQCV-RMFAX-Xc-yu59Lhb4yXJnhpWCzUMoZE0VTXC2FuM4ko6Cvpd4hnnj1XM8A==)
- Notable innovation projects and partnerships:
  - Scaling SOAR adsorption material supply for Svante’s DAC filters — a substantive move into enabling DAC supply chains and materials for large‑scale carbon removal. [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==)
  - New materials (glass bubbles) for hydrogen transport insulation and barrier solutions for EV batteries (examples of climate‑tech product innovation). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyHHuq3MoZ-a2DCV24gM18BExneS4U6-ykf1fbJbA7wZsDSAjyN-fXFtZLaTJFWr-Ptpj-jHZ-vwra6266bKhHnKXI3cfNrzlSRchkHNlje9fZzPjWgSeM_lq1NpEXb84W3J2tEEm7jjs0TMzYbFti24spWOi19M8y7O4B26ltm9oGRKtrRvw=) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHnZ9kXKTPE4osjRGi3oqK2MLS0uQUzv84Pq1cloFvvp0AoXtZF5GlS2O_QXNWI9oufGi9eSXgylmp60L6Ka9JNhh1ixvTcejDPoS0dxjwygu_pTU-i5Pp5V7Dnd2-ScfCJSvIxciDChHyoNcJwhsHBZtcpXoK3Z5i61w5o26qs5cHkVGz8_-T1_NV6xKIpoJrrwBx6FTHX0Cyu)
  - Water Resilience Coalition participation and NPWI pilot programs in water‑stressed basins. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==)
- Partnerships/consortia and external validation: membership in CE100, RE100; SBTi validation of near‑term targets; collaboration with customers for decarbonization products. [cleanlink](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNzhY3OadLAl_VPsp2IT2agW8NXGyM2zektHHDJ8XB-GzUozo37hgNbnRhUPfX56AT5snGgbLy9O1mvBjy0avSdbw7ZqGNRdRRxMumBNX0oKzyC6qm3Oce0Wo_e4Zx_Gnm2Psg78Zdoa0SRlxE1YrAwCEXq7uK5uUwRQfZr62Ntjo-xSMwopNF-c1_EtQ=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)

Justification for score: 3M demonstrates strong R&D orientation toward climate and circular tech and has moved beyond internal pilots into supply‑chain scale‑ups (e.g., SOAR for Svante). However, while 3M shows many pilots and product innovations, public evidence of breakthrough proprietary carbon‑removal systems or an extensive portfolio of disruptive sustainability patents is limited in the public summaries; therefore it scores high but not at the absolute top. [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyHHuq3MoZ-a2DCV24gM18BExneS4U6-ykf1fbJbA7wZsDSAjyN-fXFtZLaTJFWr-Ptpj-jHZ-vwra6266bKhHnKXI3cfNrzlSRchkHNlje9fZzPjWgSeM_lq1NpEXb84W3J2tEEm7jjs0TMzYbFti24spWOi19M8y7O4B26ltm9oGRKtrRvw=) [triplepundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEfl_CCdnNN-X-ADDMA3g0219dZQZ6EghrDjhnA9F2n8ZNgyn5JzYswL7lZmj1nqzRizqH-9f76cRJu3f_GlpxiG-TH9PqxZeUx7dAf3P4lVRXhMQuUhwobxU6pmUXo_73DhAQCtu8AuGpZVS1Lu3bM1a6xnUWafyiprUrcuidsA6byY55is2_bNpn2ie537Ak6q3o0)

Measurable Impact (22/25) — Quantifiable environmental and business outcomes
- Major reported metrics (2021–2024):
  - 59.1% reduction in Scope 1 and 2 GHG emissions since 2019. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF136cZEnTUa6__Qmf3XEVrXxvQLhyfzLkkYpAzlf6-fKeCdfofGMICazGr3KQLfXfWVL0QNwCKvsUmEDIY2ewErweXbOTEcjdkTKiqjX2BhVTmkyJsk4omifdcEkGhvinigsxXWMymcA==)
  - 30.7% reduction in Scope 3 GHG emissions since 2021 (SBTi‑aligned reporting). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHo4EdUxegyUtrIpxzN1TkpTCXnjGKHkEMzg10GWBUurCdrg43Ei1Oh2hcFFYxbj7TpGQiYPFcc0pu1EMJrhHlPrURrkErz36AkO5GKfcuIZhRTdhkjdhExYxiZw0EGHKx6DWX5uYDxfBI=)
  - 21.4% improvement in water efficiency (reduction in water use) since 2019 and ongoing water‑reduction targets (25% by 2030). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [wwdmag](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG60IXHB6Z7dcE8Y3gyf2qoICa8P_d2FbCaRCyP4K1fVthXP_xS7f5wsK0orGrHPp5J_Watq4yfAnF5P0sy3IVqEMkMzhQsPvvf_WZ1V1ZqRizBAzYNOiWccbsG9gn-Xfhyvu5YXUb0n9tR9U5mkxNi-eKNKS50VjXT9S1omNJuYlXQldycGG9MsDX0zfiMOfcrP5edEjCdrmGa3IkXyYH3bNVuLI4=)
  - 98.2 million lb. reduction in virgin fossil‑based plastic use since 2021 (substantial progress toward a 125 million lb. 2025 target). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U)
- Other measurable actions: ISO 50001 certifications (energy management), thousands of volunteer hours (community metrics), and reported landfill diversion at many sites (zero‑landfill at a substantial share of sites). [energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHENHk3DMsjey2XBvNkQvKUL95V6upn1IP6-ct4duvuzqXPFOH8Im9_fEUpz-fQ5mYShHWUhy5s5s6PqM8_ufbGs68WdWr3bN_uHNQvOQrTDc7VmmFu60JH6S4JJ9LZPCg4LnaB03OhlsgoU3gLVgBvczOka8fIEcq3AWADA1f8SZUzR17fs9QzOlwJE4Fxk14oNq6PZTmKVbPK) [csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH7ub1-zizj707O-E4qVMXk8D196AowJNcB54R3BMwFX2luuY6zqnfi6H1dGd6BLbDBGu3S21vlCI6ctWF5PkddCUzggFteDQ5bbSAT37b4UiDJAd21rQ-5HeGTQEsfdqcfjTBuFQUuXrMccEqtYCZa5Bpz0q168u31SpFlc23x8_1taq2Rg8gy1NP-urysMJ3kHl06PndKLF2S) [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez)

Justification for score: 3M’s reported reductions in scope 1/2 (>50% vs. 2019) and significant Scope 3 progress are substantial and verifiable, matching the top tier of measurable outcomes. These results justify a high score; small deductions reflect that the most transformational outcomes (e.g., verified permanent carbon removal at scale) are still in early commercialization/cooperation stages. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHo4EdUxegyUtrIpxzN1TkpTCXnjGKHkEMzg10GWBUurCdrg43Ei1Oh2hcFFYxbj7TpGQiYPFcc0pu1EMJrhHlPrURrkErz36AkO5GKfcuIZhRTdhkjdhExYxiZw0EGHKx6DWX5uYDxfBI=)

Future Strategy (14/15) — documented commitments and roadmap
- Clear, timebound targets with external validation and roadmaps:
  - Interim and long‑term targets: 50% GHG reductions (Scope 1/2) by 2030, 80% by 2040, carbon neutrality in operations by 2050; Scope 3 absolute reduction target ~42% by 2030 (from 2021 baseline). [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLTvrg07eloEsL4g_s3jex_BLje5-Tp9qu50oUw7mK2QXJiUHvF3pzDcujH--rX0VNvgh2NPP5ZQtOv89wiLWVvVHclsogjMLoRVAI9-TxSZ9SvGa2wU3x8Z-NwsnnXFlzm_GdlNa_-0BRK9iLCGW913cUv17pZg3fjp1ftslPJtL9fXA3gQbndxMhW5BCoFzCinxSPFFtSe_KC0CoRZYi1A_8G1NtHFUCMX8XnItl) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U) [stocktitan](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEP1A2hMC4tnCOH3CBt_BBlx2w1F1weSMSyY2BT6Z78XqSAh4BylVfkBcJWDoGZ6k6iuCD8DfzDGqLND9SwVPvvmJ0N8yuS1WKMCwZXXV8Y71IgQGziGCOcOq3gUxS57h_FpPQ5d4dRO-2MQItqPC2VkLLkyXs_6Afw9-ru_0hoHs1JwIJAkKme9JkTKQOL19feBpXP_nowMmYgK7fetO0NyApQTmFfM34=)
  - SBTi validated near‑term targets and publication of a climate transition plan. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [greencentury](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhYxCqgPPLB1ex-MsRa08BJuLP70zx-Eevys63YpFokd8CntBScMknx6OM4cFCvuq08LezeV5A2KtfURYvQUgNx1tWdIWVIZ_02rlEXrm0i8e120q_ddyQzulEayhePtQggCSNDBpcUZqdqyLzp9mNA8o_VWNjhoP1lP-eRUXGhE_m9l8fajwxyvxDgw==)
  - PFAS phase‑out commitment: exit manufacture and use across portfolio by end of 2025 (with remediation commitments); the PFAS phase‑out carries large projected costs (est.$2.3B). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=) [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMCWHzkd0Tajgvx0uVg2zmjesx3pxjqpE8_TzSsUe4ayEPNaQmgDC9aLedTR9P7hDNBLMY4B6q_NqByJ3D-JMzfwhhcaucy7DWfEs_Gs43lE4c_2OndVJ8S0lU1o3MHedZvhUWM3sCrVGlXwi5s4S-ogFsVinwgLaBODH0-mf_v4Lymz1j) [sustainableplastics](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH-s4Ka0ExA2mSfEE0-2ptg35s2ySFDNofMk2RNPaX6ihDQPCXSaTlHHXZSzqDrKZYzCGplBQk1yBm6E1BFUPqVZ-ZepRfpzWu-mXd-Xxa0k-wV3KulFVvTe1th8iYsxzQXl5WDzjQUS1bLUeJ8UzyixN_9NhRf1vCFJeDKI4Qy2mbWXl_p33E=)
  - Financial commitment: ~US$1 billion investment committed over 20 years to accelerate environmental goals. [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLTvrg07eloEsL4g_s3jex_BLje5-Tp9qu50oUw7mK2QXJiUHvF3pzDcujH--rX0VNvgh2NPP5ZQtOv89wiLWVvVHclsogjMLoRVAI9-TxSZ9SvGa2wU3x8Z-NwsnnXFlzm_GdlNa_-0BRK9iLCGW913cUv17pZg3fjp1ftslPJtL9fXA3gQbndxMhW5BCoFzCinxSPFFtSe_KC0CoRZYi1A_8G1NtHFUCMX8XnItl) [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQENwNjV7HPOQ_czx8bz-_BIzuwXGHGa0x5h7Af1VWz1gt1MMgiAl3uQZHPKyObEtygpLej6Pppy8v1Gn2BLku7AZnes1qbWBaetu-MRbiyQreSlDHwK8Q_N5lvRs8nhYs7ZNJHnGGn27pG3hwaNyYXLNs9kCg90WbS4sKubIgugW1zKrQaSrvIEZhLz8H35QBk6guq8FA==)
- Roadmap elements include Science for Circular/Climate/Community pillars, SVC for new products, site water‑treatment rollouts, renewable procurement (PPAs), and continued R&D partnerships. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF136cZEnTUa6__Qmf3XEVrXxvQLhyfzLkkYpAzlf6-fKeCdfofGMICazGr3KQLfXfWVL0QNwCKvsUmEDIY2ewErweXbOTEcjdkTKiqjX2BhVTmkyJsk4omifdcEkGhvinigsxXWMymcA==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)

Justification for score: 3M has a comprehensive, specific, and externally validated roadmap with near‑term and long‑term targets and committed capital — meriting top‑tier scoring with only a small deduction for uncertainties around some implementation details and the operationalization of certain advanced carbon solutions. [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLTvrg07eloEsL4g_s3jex_BLje5-Tp9qu50oUw7mK2QXJiUHvF3pzDcujH--rX0VNvgh2NPP5ZQtOv89wiLWVvVHclsogjMLoRVAI9-TxSZ9SvGa2wU3x8Z-NwsnnXFlzm_GdlNa_-0BRK9iLCGW913cUv17pZg3fjp1ftslPJtL9fXA3gQbndxMhW5BCoFzCinxSPFFtSe_KC0CoRZYi1A_8G1NtHFUCMX8XnItl)

Industry Position (8/10) — how 3M compares with peers
- Benchmark versus peers (Honeywell, DuPont, Ecolab, Johnson & Johnson, etc.):
  - 3M has SBTi‑validated targets, has exceeded its renewable electricity aim early, and reports large absolute reductions in Scope 1/2 and meaningful Scope 3 progress — comparable with top industrial peers such as DuPont and Ecolab that also report SBTi validation and strong renewable/efficiency programs. [6-0 summary links] [specialchem](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEasODj1SGdbCaNdZsI6-wrMVZuYL5iNScHqFYxIYzSIXT-hGlZtRiUJQFNoF35QCKUxetazcIJ0EijLDTin0iKLU3tJIzHPOfiT7z2wcOMCgQdb6L5z_fe-0aLnOb0db51AnqRD_RajWGGMrQS0VnzXR1ZdwJ23Rn3tuwtZAQQCfLr40L7ORYxRsvQTFdGf252n2viS5iOP2J2hXA=) [ecolab](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEumPC3QuNKwFZjFdo4903futNtmF6eY_hRFvjJ6mg8tpVJAJQ-is41ful-Tl8GUETLJIGLCGNw2ZZVf6faXnPyMGCkPyvQkXbnrYTA8SkgX690mJC9-KIdSSGs00m5vsQRR94Jq5tlLIvAnDBOzBA305I4QOt5lz9ZPYGkc2FpWb5-qou1HEA9RhgDAHzUdRChx1dn9YZ0XE22kOpmLkAtwUUhlg==) [dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQED-hUtWQC5CWJo_0r2JS_be-Skvy2eBIojAImAvAQDYpfIBKm-QA2y6D3mlyQT8zgyNTdyufwoZH2pp1eWyb3fVnkHLAKMDmzEBUXu-9Z5ys_a93T7q3mYE5qVqPcvJ9wjysVb21RCoDYOebPnXDQs9LAzpEO09Qt5L5OoarcuETIY-QM1tVWP0X0uI4jy70iHROKgyRM6mNSBD5tRReXwDz3DpqRI3CVsxp264xbqQHxrDw==)
  - 3M’s innovation into materials for DAC and hydrogen transport is differentiated, and the PFAS phase‑out commitment is a consequential (and costly) leadership position on chemical stewardship. [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=) [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMCWHzkd0Tajgvx0uVg2zmjesx3pxjqpE8_TzSsUe4ayEPNaQmgDC9aLedTR9P7hDNBLMY4B6q_NqByJ3D-JMzfwhhcaucy7DWfEs_Gs43lE4c_2OndVJ8S0lU1o3MHedZvhUWM3sCrVGlXwi5s4S-ogFsVinwgLaBODH0-mf_v4Lymz1j)
- Limitations vs. top peers: Some peers have publicized aggressive near‑term net‑zero roadmaps or massive renewable PPAs and in some cases earlier commitments to net‑zero/broader Scope 3 supplier engagement; 3M is among the top performers but not uniquely the single sector leader across every metric. [ecolab](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEumPC3QuNKwFZjFdo4903futNtmF6eY_hRFvjJ6mg8tpVJAJQ-is41ful-Tl8GUETLJIGLCGNw2ZZVf6faXnPyMGCkPyvQkXbnrYTA8SkgX690mJC9-KIdSSGs00m5vsQRR94Jq5tlLIvAnDBOzBA305I4QOt5lz9ZPYGkc2FpWb5-qou1HEA9RhgDAHzUdRChx1dn9YZ0XE22kOpmLkAtwUUhlg==) [jnj](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGaLvwzbPisrAHRIyHcXShFNnxumsz9qRHGeqNTSHK9MI1WZznaSpMAPwOz6pcsUMQc_FQ2KxqQfnMU-ewNSk-oaMDU5Eu9Dcwy8wqZcvdzpbLbD3jBGyNCUwM3b98IhpUcylSrgGuYe1-jhWEm8ElzwsvLryBe6IAKcg==) [dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQED-hUtWQC5CWJo_0r2JS_be-Skvy2eBIojAImAvAQDYpfIBKm-QA2y6D3mlyQT8zgyNTdyufwoZH2pp1eWyb3fVnkHLAKMDmzEBUXu-9Z5ys_a93T7q3mYE5qVqPcvJ9wjysVb21RCoDYOebPnXDQs9LAzpEO09Qt5L5OoarcuETIY-QM1tVWP0X0uI4jy70iHROKgyRM6mNSBD5tRReXwDz3DpqRI3CVsxp264xbqQHxrDw==)

Justification for score: 3M is among the top cohort of industrial manufacturers on sustainability and sustainable‑technology adoption, but competition for “industry leader” status is close; therefore it earns an “among top performers” rating. [6-0 summary links] [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)

Scoring summary and arithmetic
- Technology Adoption: 20/25 — large‑scale renewable and efficiency programs, advanced water treatment and circular material adoption; limited deduction for limited public evidence of fully commercialized on‑site CCUS scale deployments. [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)
- Innovation Leadership: 18/25 — strong R&D and partnerships (e.g., SOAR for Svante DAC, hydrogen/EV materials), but a few breakthrough proprietary climate technologies at global commercial scale are still in progress. [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==) [4-1 summary links](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyHHuq3MoZ-a2DCV24gM18BExneS4U6-ykf1fbJbA7wZsDSAjyN-fXFtZLaTJFWr-Ptpj-jHZ-vwra6266bKhHnKXI3cfNrzlSRchkHNlje9fZzPjWgSeM_lq1NpEXb84W3J2tEEm7jjs0TMzYbFti24spWOi19M8y7O4B26ltm9oGRKtrRvw=)
- Measurable Impact: 22/25 — very strong documented reductions (59.1% Scope 1/2 reduction vs. 2019; 30.7% Scope 3 reduction vs. 2021; 98.2M lb virgin‑plastic reduction; water efficiency +21.4%). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)
- Future Strategy: 14/15 — comprehensive, SBTi‑validated targets, climate transition plan, PFAS phase‑out, and committed investments (~US$1B). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLTvrg07eloEsL4g_s3jex_BLje5-Tp9qu50oUw7mK2QXJiUHvF3pzDcujH--rX0VNvgh2NPP5ZQtOv89wiLWVvVHclsogjMLoRVAI9-TxSZ9SvGa2wU3x8Z-NwsnnXFlzm_GdlNa_-0BRK9iLCGW913cUv17pZg3fjp1ftslPJtL9fXA3gQbndxMhW5BCoFzCinxSPFFtSe_KC0CoRZYi1A_8G1NtHFUCMX8XnItl) [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMCWHzkd0Tajgvx0uVg2zmjesx3pxjqpE8_TzSsUe4ayEPNaQmgDC9aLedTR9P7hDNBLMY4B6q_NqByJ3D-JMzfwhhcaucy7DWfEs_Gs43lE4c_2OndVJ8S0lU1o3MHedZvhUWM3sCrVGlXwi5s4S-ogFsVinwgLaBODH0-mf_v4Lymz1j)
- Industry Position: 8/10 — among top industrial performers; strong across renewables, targets, and measurable outcomes, but peers also show comparable leadership in some areas. [6-0 summary links] [dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQED-hUtWQC5CWJo_0r2JS_be-Skvy2eBIojAImAvAQDYpfIBKm-QA2y6D3mlyQT8zgyNTdyufwoZH2pp1eWyb3fVnkHLAKMDmzEBUXu-9Z5ys_a93T7q3mYE5qVqPcvJ9wjysVb21RCoDYOebPnXDQs9LAzpEO09Qt5L5OoarcuETIY-QM1tVWP0X0uI4jy70iHROKgyRM6mNSBD5tRReXwDz3DpqRI3CVsxp264xbqQHxrDw==)

Final total = 20 + 18 + 22 + 14 + 8 = 82 / 100

Recent examples (2022–2025) cited in the assessment
- 59.1% reduction in Scope 1 & 2 GHGs since 2019 (reported in 3M Global Impact reporting / recent years). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHo4EdUxegyUtrIpxzN1TkpTCXnjGKHkEMzg10GWBUurCdrg43Ei1Oh2hcFFYxbj7TpGQiYPFcc0pu1EMJrhHlPrURrkErz36AkO5GKfcuIZhRTdhkjdhExYxiZw0EGHKx6DWX5uYDxfBI=)
- 30.7% reduction in Scope 3 GHGs since 2021 (SBTi‑aligned reporting). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHo4EdUxegyUtrIpxzN1TkpTCXnjGKHkEMzg10GWBUurCdrg43Ei1Oh2hcFFYxbj7TpGQiYPFcc0pu1EMJrhHlPrURrkErz36AkO5GKfcuIZhRTdhkjdhExYxiZw0EGHKx6DWX5uYDxfBI=)
- >50% renewable electricity across operations (surpassed 2025 goal early; ~56.2% reported in 2023). [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez) [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQElWOquPk2yFaN5miRnh8karS5IYgVhWzh_0-xGiiKE3t8nfPR_Sl4CNLo8_KmKcCmBMOo1kGGNSemi8st29e2te7gb-E8AmGZtTZtq1-2k8BA8wym7lnL-IldYVLge7RfP0vStNoVKJBC49G0=)
- Scaling SOAR material production to supply Svante DAC filters (2022–2025 coverage). [vietnamnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==)
- Installation of advanced water treatment systems at large water‑use sites (granular activated carbon, RO, ion exchange). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=) [wwdmag](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG60IXHB6Z7dcE8Y3gyf2qoICa8P_d2FbCaRCyP4K1fVthXP_xS7f5wsK0orGrHPp5J_Watq4yfAnF5P0sy3IVqEMkMzhQsPvvf_WZ1V1ZqRizBAzYNOiWccbsG9gn-Xfhyvu5YXUb0n9tR9U5mkxNi-eKNKS50VjXT9S1omNJuYlXQldycGG9MsDX0zfiMOfcrP5edEjCdrmGa3IkXyYH3bNVuLI4=)
- 98.2 million lb. reduction in virgin fossil‑based plastic use since 2021 (progress toward 125M lb. 2025 goal). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [2-1 summary links](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U)
- PFAS exit/phase‑out commitment by end of 2025 and remediation/class resolution activities (large financial implications). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=) [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMCWHzkd0Tajgvx0uVg2zmjesx3pxjqpE8_TzSsUe4ayEPNaQmgDC9aLedTR9P7hDNBLMY4B6q_NqByJ3D-JMzfwhhcaucy7DWfEs_Gs43lE4c_2OndVJ8S0lU1o3MHedZvhUWM3sCrVGlXwi5s4S-ogFsVinwgLaBODH0-mf_v4Lymz1j) [prnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFgZ9g3wuOXrGZ1I4Ksb8sHbo2ll6dQFm38FQgd3Uf6HwpZe2fBsACHT15d2erxWvupS0tg141iT_6kRNU3_NK4R_1fEYTYY_z20uSxSt41XvrI37hFGGdJLTxK0PGd0CHQe4oksSXf4_nVBY841b77xmG0rdixw3hdnU30x6IpQNmiLwTr8ViESG1_xE5evZleR_e4HujaGq1Z-gAfYKSarfkh)

Selected primary sources and reporting used (all citations from the summaries)
- [able123converting](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQElAIp0iAVP5mEVwwz8mcfRcAcJFiMGWW2BB9S0Oi1uTQtwfM4_vE3gnFQIjMV2OEmX-Xyk06tzkkZcf_cp_1jl2RR8NEfF0ditTOB0AJKOBaSrFsjD7Mp0G1jcwH8lbG7A7jpmqpljXgPIkbhRcb8vShSy5zUzUZu3UD-s-un4_fqLVvxv1Ddk400srz6rSGP5Va3cQNfi5rr1TWW8PNkG-GIUBjzB)  
- [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFWYWSQt7NTfkGdUyHZAshYM150I8SBQ9zewkFwXo0aoiWoIJQPHAFveZDKmQIskNNqvwdhskM59St4lfC2SQ_Q3S_rFgABqGIC03pjhgedupSkj1dj7QKrAo_2asT4azHeMEDk-UQx3KjZsX5JVxNWagPbAzQ85JFb9NL3vkg2fP6SvGvNoe8i9iKbmYWFTCunXtQghQ==)  
- [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGF7ediy5rSNR791YstNtTaGwwPJdVyHtzcNmhvC4Q1X_NU9rlQLo0xXquBTqPmW9l7DxXLi6GTUuO9HSHZxmWt5y0fgFD6SGhFHZmTHiSw08wEfgAED_Jxa9JZxe9XWaHBZjplYztqoF24pvCKnNBCsPPbVmptVwwUdfiDDCTy-1xRC8JCBZpzTkYEb-1U0YT_KUWSJCQhcslW14V3IxtBIjxQXIAErjP3w7qLjhmn)  
- [researchpublish](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG3qMEsGAY-ueGZ9h-WDH7Gx_GDgv2PNDtHPzq5bu99e01lCXhl051PFnHwLFMwa6v7OPLUK_E9ELjqF64c2ppehAltr9i7Xa1B63gZtiwmVaKRQobIneXq-X1fiv4wrcjEb5hvlV4yr9kEL3eQ5ydLVMJvEnT_606YWqEUPzyV3qzPJEqqCjgZC4JIKxPCUunEu6pONlvo7NuV5_QAvg==)  
- [sustainableplastics](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH-s4Ka0ExA2mSfEE0-2ptg35s2ySFDNofMk2RNPaX6ihDQPCXSaTlHHXZSzqDrKZYzCGplBQk1yBm6E1BFUPqVZ-ZepRfpzWu-mXd-Xxa0k-wV3KulFVvTe1th8iYsxzQXl5WDzjQUS1bLUeJ8UzyixN_9NhRf1vCFJeDKI4Qy2mbWXl_p33E=)  
- [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHMCWHzkd0Tajgvx0uVg2zmjesx3pxjqpE8_TzSsUe4ayEPNaQmgDC9aLedTR9P7hDNBLMY4B6q_NqByJ3D-JMzfwhhcaucy7DWfEs_Gs43lE4c_2OndVJ8S0lU1o3MHedZvhUWM3sCrVGlXwi5s4S-ogFsVinwgLaBODH0-mf_v4Lymz1j)  
- [sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQENPnyO0AnLbabvldpbImA1RtlQrQq1hLslBlCq2vUfVseiqpbNZdIbD4a8hsniTsI8JqxWQWvcaA553faffJ2TznzMqj_UJrdrU5a0_D0BovosUHz0ZluBxVDobDLZ8AFNtP88FL9F5tMjoMx4uWz72pua8-DjTdKj2sbn1B6mvcntY_07jAPHXNiRvIgSAv52xQdJruOrwucMCz3i4jRtkJEiWzFItg==)  
- [cleanenergyministerial](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGsk629CJsAR7WVyXOj22MYsx6oyWuo8b8TRuCTVKmDJWyfRFlsya71XJuzvAxsUJxL8HudC-QxzBfKhj1p_vOL1O091wVWHpNX-CZGWjZV1lYqGSVR1eDwveAVeJKjp-vLnDp6ETMy9y4LmFivflv7j0rvGFvN0YotrPmLdvmSNV8ELpi6-kWx7oxWt8CV2Uar1If7Jav-3Q==)  
- [3m (sustainability pages)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHE2ELmSbfKi3-Ws3R6x0Yo6Fv8pDyjRDqpBbzPK1ELzKc3hOPB1gddpFR6PTfLs3tqR9dNnC-T-BLyEkNWk6xx1HTvNfOiCC-KWfo3N0_qbnRk6CWC6QCqtQ9GsbxfgoaWO6IXU7ej2bN5gj9DGXBCn9dajM0HWhoIQZ9SxfbqsCSGS9t0HGHi)  
- [trellis (alternate)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFH--4_8tDqzd6rpN3T_HL2LRz40pEETa-VGuFyFyQm51rxHOyBtIREOIBGtEkX6hjuhHnJgdnqHVOSxEZeEMj_QUcVnOrVRtwwBjDI5iS0jD_Q8tG_ZGvkwcy1HyciNDNdjYD7zE9i3hTJKFsyFUmjYCBBuGJBmJt4Cnuk3O6inZJ-OoLxaIARsX3pdxXq9x62mFYWNBM=)  
- [3m Global Impact / 3m news](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHUhWoDkQFq3CWwlWJN7NHzuvkqJw-Yo3plBSppQPp5jFE2gR64YB8D7ZQjqOrmnCtmOU6YrT2MHQhHn9ixonjpdKPLuykuYBjUgy8szyvlsiwH8jI7aydff_gp4wVNCVv0xQHbguJTHPo=)  
- [netzerotracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGp6E2T_m-fV5uf1ZDUkJepZdZ1E_WTK7jFXdPcNk8MnR9q7Z9lR0_Qf8e6N7RQaE3euTVWSeVbAEG_tdW-oKcXU63Lh0aXxUUd6fUQA2fiIVR8md7d95h2gZAx2V2IYOxv-j0ONPg3vmlGTHIP205a1VTmL3nfo8_E5oitgGRMy-7vzQOODQ=)  
- [thecsrjournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFrzJQ03Gj26AdC7IWkbunoFM80kTUaxYJMi7-I3p5CFd3LWG6doXOIYgjyJPC-Ng2TSmlhBntL5jgeho1akzPF2CqG5cJjNe8lVlWC-FoJPHXehIjamwqsxhxB_31H0FUBJYe2F7QLGGcfuGjRFaMgXuJs0L56LCC8pmqM)  
- [triplepundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHZqFG5A3KZodQs0yMPaKhJgOwb3JBJwiD_QS5XhaDSrrgqxkFCKUn0vpYqnAF5wJbw66phlQEnwZiJ4QUl5vZQzf8J7Sao5a3Q1BnQJv9fzD53YOfwTJeLSl5-Zjlz-LqC6jzW0MyZCo0ch6SU1XkRJ8Gn4Z1LGfwgCWZ3bqweSTdpDsYKR7iqZXfjjkgToKXKn3FTsQ==)  
- [wbcsd (repeat)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG8QLqGq6kKZ_uVIE8j1IVkWsAacJpr7ylD0ox1QQWZ1-zvB2Iz7XLbvDxzFHTVG6wmw6jNf7J0eqQ4nqO90eBAyi5OMOISbTZkOlwhQGD95A8QArIg6URcxJav15DUy5cKYikK1ajHjkQ3RCeiLV6xwwp9sYq7OaG8bC-fPcXVOPPxjg3tXRv8gwjnqO5dmdwhN3Ir6nN2orU0vf1aQKqrrB4mNuztHeM9amPCDRQ=)  
- [esgtoday (repeat)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTLtX0jnNkzD6F3p4iXyigrQmtVz3yfQbOo4NbuVW_TRTOfcPUEOf2Fu0Hh1BIcQC9SrVvRs9xupzY1ZUXp_qjanQTHWApo_hKJcSQo-RTiwoGp1U_xUfkEBvShTXVlwNEx-3JaOuJenQufy9Tg08AzNQp07MGmn3lncnJvh6KPMTTlB1f1riTssViw-MBP-s-SXsi)  
- [startribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHu5HmHwX2vevQu9FMmCDiwaj9uCzf4XvqhnbasNSoiOX16QkOLyFcke3gXBF2jtnx1vD7prSiuvztF4tI8PBuHCCw7SPvW9slFJIKrNym7nlIMziq_QGxJXU7xTbNu6BdPcTKdBYUK3vEnYtK-n0jft_Q33122bJ-TvJSUUGYJ-dE5II4h4auNL21SUzEWd0rb2i8VKx7rHUqvLMFCeZM4CskC5cFJI-xCO01aGkD-VHfLOeCf2qi-9ZIbbENA)  
- [newswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLaGs7ET5-VJ3h3PMfY5a98RiKZdUUsU_Lldbfm49fmxcTHnBWkunC39RKHFamxScDDLngo4wtCwFrSquKv8fPdARz0AGKZRes4fFCfyqQK0gwy57jnOpLnX9fJxk__cGoaXuz5u3TE-7749sTUjW4TJ7eX3Xa_aG7LcwfZrPPgh80l3LjKil6ZzwNZOgi52nj1oNGS2CABxuOPUXB2Yz95nuz5XSKxz7bXDBPcTtKqLM=)  
- [3m (press/SBTi)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGzJQQqq5t3NcVNT211X6BJI3td7Sm740pKw7I0TDSlh1PWTQMmoWze5sKsrymmxmmjubrYDpxc3XuKArrEIWIqVDxHDyAW9qwhekKGPMpgEJaNi1y5glJI3-nFE9NYhYuZOcXFM4GNmxKSIy-MtaQC9Xb4CQ7yGRU3IbBFsryB3jq9UGcfZyI=)  
- [startribune (SBTi coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHVkTABe9OV_URl9sWlHu91eG90IRKxh792dnZHlYSdRqrKdAsYjmrh-E2CQVxhfOnBvHrjCyTA8-SRG40gil6aYQI8WhY-m0n1ZTicIvp5LpLIe4f6b_nGWjO9_9ohArKL90vrTepmmBX1Nr9h7eA405WeTUbHvKg3AcRU99HOWQpnq7QugMx5bIu9dP8mINVCFzU3b1LWg8eSMGrlyA==)  
- [3m (metrics)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGc-oLxsUvVHK0ZJwps9_trtT2jnIBNALM_jH13nQFq07DEYZqv1o_7NYvQ_f6HIgO2JRY9FUO84P2-h8xFeyWXH5jtRiXaBDUhsJxIv2dC6YQOAKf-RggywzLHsStv03vpuCrsPNXw5Q==)  
- [reset-connect](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFo7Q3Lk0HsXTkFNqmYxNmjsoZDhHmc2SERsNd4Pgbg-zFM4FPYgGAzMn236zQHC8zCkBXmd1jQrRgjWTUCwtbRGtXn0f0QCEYhqkeLFVDyUGnN_D6q3wsq285KRpCX3SdAP1hIbg==)  
- [able123converting (water)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE01KwV_60BPHw7KGZnWm9hNhoiKP8u2GLt1iDPvswzypcAfNzI1kZQ8r-RHpcxUatWmcM2AwPP_RzJBuJF2iPsfka6XhQJ4RUp1H1rBCFZRYQjW10MJWiW8DWWUYtgy60gbtkMlVIp6G-BFGGnx58xWzDTNAsVKwdGajmnkPSmJihlYtJn6qrM5c6xsqiqq2sB7CfHsif_S7AQqacHFNA1EIqT7JM=)  
- [sustainable-energy (renewables)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE6M-Tkep9EJEfZEEEUH6ONV-vMXW2nntR9JFstwaeaJQPjeJMbbwaOiZlBDlXDLtMqX2fjbbphg1zkmJvqf2Wjq_lyZl9E_Vr6pZxhxIKF9DvOvxLILDCdnTppkQCUGaFIm-ngtbWPkZZCUqKJEOcM842tcaTeBOgvRz-er5j0l9hd0Z5yPG-vF7Lv2OYU6oMwvajpRzBnAzUn2JbD3HST7n2usQ==)  
- [csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEQGOpXyfr3emYF8x9etEbTT-DCvOtR_OiHMas2rda-6NQl1lWwwGb8OnmsfleGcyLW4szFfCkaPE4AMCBGYn6k8l1DEN7ucduJTdb5ouFYQg4loXaQZ3An82HrF3f3uaFCcfpDHQyvivab6C599-UPqVrGYPzb7EEWQ_XuyKZTvL2XZVxv6magzHGB4b1xmhgDzyFZoYMAEMM=)  
- [thecsrjournal (waste/plastics)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGWhc7Tef_XY7aWM_uG15WF6P2pSRlF00aEwAJrx_mbWfy2Vc5OKebktXDSwV75JkgU8IsWuJUb9tSSuDStx9mScUDjqGF8KUsb2VraNi3s4mhbwHgrwvZ2Nt7F2hDGD3YvWPIE5AmFAyPW6PZKVrJV7UvoRICpAUXl134=)  
- [issa](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFElPZvSUtp35hylc-A6EbPpKci_wSVj372Ls7yahp6QeNeQIWSj3KwEL9qJGew9m2400mV-sSsN0RAm6HZvVZKAkJjcq8edqYvOYLziOMJpQPh_kILQtabKEH_XnecGXC77Xn31rCqhbubuK2zJVzjcPX_lNc-KaBo7QedEoo7iWTa)  
- [iowaenvironmentalfocus](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFHg-v34qB8FJy6B5efOUffI1VCIpUXMZdinZeP9CR6dnVbGIOx2LFPZzpYsPCVyjQ757htoQCjxB3uz3QPNM20NZncmrwg_JD8XuQ3aL3qLqfAl8dMOQSLvVfWgWZndls6KHQFZ_hNIIcXKZYOR_RGecvi-folyeTwZgGpo5JKYXPQnB4cQSxueZtVtAhGK3wC4ZvVGA1XFN2V9mxxT-U=)  
- [trellis (PFAS/legal)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLLlho0318JsgTqOgIhugRU33KkMewvpD5sWQveVVeMdUA5xIwL2Pz0M2tFAOZnKatpnrq-0EQGCf8NxitqeQMpA9vDSO9mt4qjUy-XcldoEH0VxzjOJ3jS3ZDjdwFdSotEOCdNRLRkBZ5GGvZFyPc1Ytst_ZFHEyM1uB47Qgn19AnHVs=)  
- [3m sustainability / roadmap](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE3xLMTMjoaEVcsD3fdvQ75c-CvFPsWwKC8qOshKDc6SwpHs1vxG_zzelF-KdtAwKqgJdx1SlxbE1-leGps9WY6WILd44HJpMrat9tpdrdiAcUb1QuPYdkDSeShR2vPMxuM3fM0adBfJfI=)  
- [3m updates / SVC info](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFTmUSQtimq_nvQowFhv-z2whUI9TR_lgIwM58eMRSRoFfg2OdLiY7q00NIL4sasxsyGHlinuTOhGzKBt-VgXqEFvEZ8j2Ta0exS4am4OIk9JW_leXEm45ZK2WFh4NyooCOSgkv4Tsk7w-vu9xnwcAOYOB6UZoq_kQyAWMEfeViK6nrWeMnuF4U)  
- [wbcsd (roadmap)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGLTvrg07eloEsL4g_s3jex_BLje5-Tp9qu50oUw7mK2QXJiUHvF3pzDcujH--rX0VNvgh2NPP5ZQtOv89wiLWVvVHclsogjMLoRVAI9-TxSZ9SvGa2wU3x8Z-NwsnnXFlzm_GdlNa_-0BRK9iLCGW913cUv17pZg3fjp1ftslPJtL9fXA3gQbndxMhW5BCoFzCinxSPFFtSe_KC0CoRZYi1A_8G1NtHFUCMX8XnItl)  
- [greencentury (climate transition plan)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhYxCqgPPLB1ex-MsRa08BJuLP70zx-Eevys63YpFokd8CntBScMknx6OM4cFCvuq08LezeV5A2KtfURYvQUgNx1tWdIWVIZ_02rlEXrm0i8e120q_ddyQzulEayhePtQggCSNDBpcUZqdqyLzp9mNA8o_VWNjhoP1lP-eRUXGhE_m9l8fajwxyvxDgw==)  
- [cleanlink (memberships)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNzhY3OadLAl_VPsp2IT2agW8NXGyM2zektHHDJ8XB-GzUozo37hgNbnRhUPfX56AT5snGgbLy9O1mvBjy0avSdbw7ZqGNRdRRxMumBNX0oKzyC6qm3Oce0Wo_e4Zx_Gnm2Psg78Zdoa0SRlxE1YrAwCEXq7uK5uUwRQfZr62Ntjo-xSMwopNF-c1_EtQ=)  
- [sprichards (CBG sustainability)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0olJJ0T9paoXd3eAYc29JHKrT3K9g3_APqZaJZ07oKAJBgaEFLFLZoQbrPmxuHmlY9nKR5btV_-MH03mW8N8CjvLOV45hK805PIdvWFJfh8Ff7nMKibHiY5irgOzgbIN7C0suWgJkuWjLea4OMKSpX04A0tSWqLYATwPxh3Ug36y96gA1AlE4yqyEWYTI-XxjlRpjigj6iZQRKTAWW3Uu)  
- [3m (overall sustainability)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9myU1AmzfLbcj1rjf5am6-HoZ0YvQCI_I-XMlb9LCqORGgJg85nO1jurMxVjI5h9i1dm-MkcQbx_Irhkpe181dXeeVtXxBWtrGn7vQmpHgJzyynRMZmLt3AVgtyvwui-1nW7UYiuuwxM=)  
- [3m (2024 Global Impact)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId)  
- [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9qY-WTX4zMinkhojyO_KC2wNCVBuG6wgKMNh8Pvy2sXIpfabh-1BmhTptu1OsRaKrbBfr5DGvxEg_JXaGqBVgn2CSxx-qKf1434utNlHcODDZxEZzRz6X7bejchwrH5eCNG0u435JipfhKZQu5bfvM4Zj8aQ_mbdv7ADrrLcwWvYH7_P1ABBo51N-wgivyYQ0W_JXmbTzlMHU6w==)  
- [reportalert](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIDTTyQH6y8XWFPnSRbv640N54TTxT4RlZ90POg_NpG32pbCMkmT0Dlia_ldUjbqVdZsRP1VrjLST4WrgzG3b-bxeAbdaNgyKN6y8PlDgp64HWQsGLTW6lfvrRnq9KcM7vHleRCIVGVM-E7WiUkIDgtgMzetMwJfFLua8_Zak0YuEPJcmP8U85Yp8tg3QMx5lHULez)  
- [stocktitan](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHynEGjXXyd73GTl1TeynO1K0JBLYwhw9TGtZDMS__AzswoMBao9YH-JtZwTNWOFdpqR6KWBqQOW5BUh2UQ1-bf_mIly8rPJ7OcR0rNl7vpdSQsHUVlkp8aGZH2NFbJ7d8pjWW180tbdnFjzMruO4PJQUI2ChqhP57Lgmy2EPuSpYBXrcKyiPPUAaK1Faah-ldU8P4S1TeipYxKw_3v)  
- [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQElWOquPk2yFaN5miRnh8karS5IYgVhWzh_0-xGiiKE3t8nfPR_Sl4CNLo8_KmKcCmBMOo1kGGNSemi8st29e2te7gb-E8AmGZtTZtq1-2k8BA8wym7lnL-IldYVLge7RfP0vStNoVKJBC49G0=)  
- [archify](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHru98JYM6ydYq16YRGQMTphqBEbUG7EZY31qsu4YE9sSuoN0T230TcsLCDdDzU0ZwQcN-JeD6s3szxuoJR0tvvT-9MN4O4_EUAAm64roawhi9fQDc-TGphKFe_KqXl5C6YMKTdAimm1dmzcudGj3x0qBZpGXvb_pOG0AKZ0HqW1pAU8qL-7X7khSVKsBr2Ceve_SzCvClsk-Ok14EeKR63OTf4YA==)  
- [3mdeutschland](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbCkLV4nchvqUpPJYzmWF0kiacHHGp3IwwKKtDkCZWWE2_0dCKUvR_2wLWB9rdaJl6NFIoWgXZEJFXasi63w7ppSHQViDbgoJkifrfMf9ueS383mvhViQmLEOfFH7JB1-614nCjQbwpVuT2os=)  
- [rmi (Gunsight PPA)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE4LGdy9AQM24Z0ZgJcTijxY2jMgYkGvCBlcrB5Q_ZXVnFX8YqhxLekOjIlAI6XJAsT3qP32LB9JxK6PCw_q4_Tq3CNVYTHqlZQHV2snJtRxQbjKoRm7PQPptWD8NfGWUpNspkbigblJXVkE3Ssiq-33lVJCOnJ2tzlSyBZWz9CHcWcGcwSKWqKiUDcfuGS)  
- [thecsrjournal (efficiency goal)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpPLE4IjUWq3i_PbMuXOwKuzoatb8l-iGlwjJ4fMdlfbe4Ke_1-ji9R0T79H5k8sgfBqk1Db3lJB4_NyENVvPNg3dvXmzSmk2t9grtmzpCsdDSbDpSmNv0KGVxCpu-GPOpM_67XiCkusAH6NlLfzjeqKgqVIy32CNCidbB)  
- [energy (ISO 50001)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHENHk3DMsjey2XBvNkQvKUL95V6upn1IP6-ct4duvuzqXPFOH8Im9_fEUpz-fQ5mYShHWUhy5s5s6PqM8_ufbGs68WdWr3bN_uHNQvOQrTDc7VmmFu60JH6S4JJ9LZPCg4LnaB03OhlsgoU3gLVgBvczOka8fIEcq3AWADA1f8SZUzR17fs9QzOlwJE4Fxk14oNq6PZTmKVbPK)  
- [energy (motor savings example)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbwHA5WshV5MMkvKM0VIPRkhcAq26xVMvjc2G5bu-6_kuNdPz2Uj2U-JwbHXt5964Y-LoSPLqJkbWMuQhF36HY7g7JLsF0T0PmaX5p6_IoH2uXofK5SIA7o75X9EJ7w6S3qj28S__NrTCii-u7BzMwVh0GtDaQ65jg_P4=)  
- [wwdmag (water investment)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG60IXHB6Z7dcE8Y3gyf2qoICa8P_d2FbCaRCyP4K1fVthXP_xS7f5wsK0orGrHPp5J_Watq4yfAnF5P0sy3IVqEMkMzhQsPvvf_WZ1V1ZqRizBAzYNOiWccbsG9gn-Xfhyvu5YXUb0n9tR9U5mkxNi-eKNKS50VjXT9S1omNJuYlXQldycGG9MsDX0zfiMOfcrP5edEjCdrmGa3IkXyYH3bNVuLI4=)  
- [3m (water tech)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtscauhm3IcEdosKPw2b4v20kevlyxuV5H8SdGczUpxpn4ZMyBzaPW_yOiuIeIODebDzWzxaQbc3ZlSOog_7sODZAcHFM0xXKPTeH3PvkeRYJ48st5brXmKzPjJ4b5hCn2JqJuih91Dgqd3OwogM6rjsxDpAtulECgACt5-oI=)  
- [csrwire (plastics)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGc01DlTvg_ggSnpJS8UvHFl576Nr6FTO_HCB6ld2cNyaowk1dGivbGR2dhWIGWtFBb1kd4gQj_vH7gwXEOumqYDuWOw6Y8C8nCD3N1miy-mDhBg1C24bk_dthfGmcUivmRryMvMaN8Qzsv3bKHnWRtUSA6K3XH7t2D_Ycw6WGc6THA1mhDtfjgK9PvMLT2xMixopgieAkruxin)  
- [prnewswire (PFAS exit)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFgZ9g3wuOXrGZ1I4Ksb8sHbo2ll6dQFm38FQgd3Uf6HwpZe2fBsACHT15d2erxWvupS0tg141iT_6kRNU3_NK4R_1fEYTYY_z20uSxSt41XvrI37hFGGdJLTxK0PGd0CHQe4oksSXf4_nVBY841b77xmG0rdixw3hdnU30x6IpQNmiLwTr8ViESG1_xE5evZleR_e4HujaGq1Z-gAfYKSarfkh)  
- [soci (PFAS coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFPkoXykjiB-n-Z8Vixphedh5dv2kriubukS7htZrRuy-KME4qk6iJxdSikMgxzf3hb4qnc6-AY36O1DrGWfpf-KRVLO0iyUch74MHzqZwfez53eAGm3ZU3KaJElQWv7RICPtE1mujkVEf9k5uloZZ4WYV4dYuw7TA9MKKZp82shxWI6SPdE-mY72vjihtFkbg=)  
- [regulations (press)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKSTqKPyQ4EfXUiQCcmRf3Hq35DxDIMj9IfdfHt9jX75-qLUEAbFlCTgGsq3kIlcZIUSqJixBreOMR9qBVdEyT04NkP0Ai2XZwANvemtmDrsblxPsUyrd0I4cYBE5dq0ojsPa1Mh_jzVbxxC38SdI7fyPDEMyyHfUmVDRyLq56xgNnOhNqew0=)  
- [3m (reports repository)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErFZ4u3qdz7NWRrWSpq0jT9AvMRBGuQzLeFyMJ0o5D175HtXqXWbNNIwAoIm0QcIugqrIeQ8v9R339JYsfyZbE_9E5_s-Qu1SNtg5MwgIY2BnR_jotUJ1KUyFDCLJ0f0GMcGFC4grHovtRbOlyhvliLBQwmDCpi3sBi8SO_06HOpoG9ELpzJw3by6Au_Xx5xZaF2888dyEN8kZzA==)  
- [accessnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpS7LS4dHzAMRmrbdqXrOymxJjsSX3nIdbsatxW_NOTr7x1GhxSewx8lT-xkUXN-6cpbSvaoI0YUMSVj0ttCQI1e17YUBzxyoZqbH6QMjwfE_GBnfv8xL6mNK2ca8BylF2KNC7v5bbAlJnenQ4wB313QXHZK15y_coK1CMBeVHG89PoB7NvbpEtbJfR75W9hmVx2uhlgAGtKBnsAu_mqGeZ5A5OHOLd1R2EMguGa2aEv50NbJ3l1uaeMKZXRVESN5wC8N0SkCparVurlo=)  
- [scribd (reports)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNJ6VBQg4FrBZAm7_1l3wP8W32O8KChkZSIqTHm_sSNddUhOI_owGGDwsS2wPqH0GQwUFZ2qdO-E_1AfS0U3YLlJcfkmBh7R05C1swOhlSQBjcZP5_EsGZu5MVRuCl1JmVkEnpXvlIu-ZVacmWgWKLFIx1dIcgMdXj52Ps5Zv4NN4=)  
- [3m (SEC/filings)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE65KgYExf41CJY5jTk82WRLC1WM02kxbrMZPr4O1ro5RgEHc1Sa6ti_GqJe30o185dLa-kg-3HQJbipeTxvVtlrzcKoYU5_5-L-x1eG7Z5W0B28FK1Ps5kHelrKiHCiRb1Z3DvgDcwVuT6)  
- [sec](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcyWj5gZLA_t08NtjB-HbTyiV44d8-KFEkiY39Y80BLcApQOi2P1uefAgzZliKvK9Qe_y8i3fXncPFN9sfmxxoZeUiGPAeKWU5Fao46-EXru5gGYEuMgQdP1hCRU_81RkpnSR9UPSfrDQYoeNDbHu9ZC1u2BvZ2aEsr_agP6c7CUpcpbcm-UjkjFqWSzZw_5Ky)  
- [3m (sustainability pillar detail)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF136cZEnTUa6__Qmf3XEVrXxvQLhyfzLkkYpAzlf6-fKeCdfofGMICazGr3KQLfXfWVL0QNwCKvsUmEDIY2ewErweXbOTEcjdkTKiqjX2BhVTmkyJsk4omifdcEkGhvinigsxXWMymcA==)  
- [3m (SVC / strategy)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyHHuq3MoZ-a2DCV24gM18BExneS4U6-ykf1fbJbA7wZsDSAjyN-fXFtZLaTJFWr-Ptpj-jHZ-vwra6266bKhHnKXI3cfNrzlSRchkHNlje9fZzPjWgSeM_lq1NpEXb84W3J2tEEm7jjs0TMzYbFti24spWOi19M8y7O4B26ltm9oGRKtrRvw=)  
- [eriks (SVC reference)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9ZxRxkU8-VMdJWQse6ljW-t55wv9xyAAlUOoVCSAdBDWa6CT3oBWpdIbQyL9Kvlin4poyX2JINmJ8dRHc2bN19tdISGSxYkZWhRFCVDqsiIXeS08uAjxZH3mNSFrbmJP51brYjNOhXB8L00LuQCV-RMFAX-Xc-yu59Lhb4yXJnhpWCzUMoZE0VTXC2FuM4ko6Cvpd4hnnj1XM8A==)  
- [vietnamnews (SOAR/DAC coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFr5enEnfTp4XIRFftWT5lTRJmG-Sq6IGtH36FzQqP_bCkwWy6qR7n1S-8tBERwrfQ_LG90L9aPCF-ldV8lQert3MOnyCi7xoJmrAuvthcWqmG_BTWbYhR6kOvHbdN3t6zlcW5Q3PfTCC9aryrslmTg1wPq9JK3ArC3lEHU9kcbx_s6pmY1aT2feL8d6Czehw0MJn5ihe8TEQ9AWNBqvMfytycgeBf_HbSsfOCPzHkIRqyfLSYy28e_t7rsUpGCghG0tSmE6vgONMcczujXtxhr1A==)  
- [responsibilityreports (R&D/climate tech)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHnZ9kXKTPE4osjRGi3oqK2MLS0uQUzv84Pq1cloFvvp0AoXtZF5GlS2O_QXNWI9oufGi9eSXgylmp60L6Ka9JNhh1ixvTcejDPoS0dxjwygu_pTU-i5Pp5V7Dnd2-ScfCJSvIxciDChHyoNcJwhsHBZtcpXoK3Z5i61w5o26qs5cHkVGz8_-T1_NV6xKIpoJrrwBx6FTHX0Cyu)  
- [triplepundit (partnerships)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEfl_CCdnNN-X-ADDMA3g0219dZQZ6EghrDjhnA9F2n8ZNgyn5JzYswL7lZmj1nqzRizqH-9f76cRJu3f_GlpxiG-TH9PqxZeUx7dAf3P4lVRXhMQuUhwobxU6pmUXo_73DhAQCtu8AuGpZVS1Lu3bM1a6xnUWafyiprUrcuidsA6byY55is2_bNpn2ie537Ak6q3o0)  
- [sprichards (packaging)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQET-mW59iMIq_k5WnGr_0hyFsQZOaqW9UO_66xmFmkTz4L9vqoJW7YeV09TBQajI-0NLJ6UAc7wt0EKk406jk_gOGPCtUGgNYmZx6GA83qpCeQnWoK_vXcU-w9907o8Ze5H4tPGM1ac_T2k_1kXBOWF3BsGMcKzSLDZJBI3UhtIb3HKVVLC1-QUdJ_yjMF5Rk_zj935nC9ZJpb8pxC9RT4=)  
- [wipo (patents/search)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHn5cs0WkcP_CoEPdxOAxvTJrlGYl1343Z0qZofhYlY1AvEtB0SQSpmE7p3-ouFKPvGO5_czNonmWxxhwM4T_FqSjw0pIRF-IYoupJCbM6BcKD551yy6VkNLdasItq_R1gN)  
- [prnewswire (sustainability announcements)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0s0n8BAa9yVni-HfEEf21XiS18tK0LCEJ4zoIH5o3e_cLDZNlgfVVb6ZZhiQ5GYY1jG38AcMNzjO9wWGCUMRyOyT_jtH1tUUXJF61Jcdk1l3_mgFJlIvtYOAgsDvbxta217g5qDQ328xDY-i2mwnOrqqXl6r7LG1iWndeeWofw91JSAtAe1AYZn-496gxcx_zRZ622iGqrsVm_swivAAzGcQifRVP2PxuT18RrMlYh3Tfpr2OvAV6iQPVkVg8XzoO386zdmT6cYW7-TRMjZ7MhFw=)  
- [wbcsd (3m coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCaYOlE4Yk3_itUkEoxwwTOgNZ5e9FVK0msbxHBZb0z9Xtm3ucgj9OO8HZTYvttVMu-iIjVXrozaAOcky4aFfTLN9TspIspe-XBuFc9WAvSKSBTwcsXjhQdz4QcOGDvvJokBSOrw1Ds3y5WH6jjOD8IVnx_eMTqCKqM8m0yGUvlndMZnZ3zDCVXeBe7_zlOA9kMlpgiipgH11kXT6e0CKyb0gOq56VNtdZ1r4keEgS)  
- [researchpublish (P2P history)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGfGBSyNMtKUjgyi5BA8bd3z3R71m2If-16fQG8Zi4GZirQKV7b-nBh1SolkrYyXwDfSg7ZwbEAOPEqd41WJT-pa8tCtn6HX9OVT7Rb7u3tMARYAJJO-BT-m1NLFH_sYGE2LBM_9KST6mn7rwrLc58b0Ns66AEVD7rSW-w1oiIodI49LcAqF4uXxKJt7IzTbFoQxbs8Taw0Mjsa5LLEFQ==)  
- [csrwire (history/metrics)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH7ub1-zizj707O-E4qVMXk8D196AowJNcB54R3BMwFX2luuY6zqnfi6H1dGd6BLbDBGu3S21vlCI6ctWF5PkddCUzggFteDQ5bbSAT37b4UiDJAd21rQ-5HeGTQEsfdqcfjTBuFQUuXrMccEqtYCZa5Bpz0q168u31SpFlc23x8_1taq2Rg8gy1NP-urysMJ3kHl06PndKLF2S)  
- [3m (metrics page)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHo4EdUxegyUtrIpxzN1TkpTCXnjGKHkEMzg10GWBUurCdrg43Ei1Oh2hcFFYxbj7TpGQiYPFcc0pu1EMJrhHlPrURrkErz36AkO5GKfcuIZhRTdhkjdhExYxiZw0EGHKx6DWX5uYDxfBI=)  
- [techsciresearch (investment)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGb76a1AA7kbdWqjRaGqnR5jW2v32RPydpP9FtRLh9fYnHl4RvP0w6JQZ-nIgJjt63cnysl-giyUqvtstG-WvATZHLtkR9ztgGLKQbFknMntVz4WWypQnP3t8wrcjbHal3IiBor4pjQBsWWsIETRB_CHDbkEpYhWNxSM4y_XFn3mIFFQR1OxlTUCN38Ay5MhSwUDVtHR6kdS5tiszTahqDeWxjmpWLpovHj9IYSGXwmKoYqQXBN_S26mml36hcYKA==)  
- [wwdmag (water)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFWEApu9raI8nlC6VwZ0ENoj7XQuNVTLi3HjJhI7vve5_v5DsY83RNeRxtFzVYBQlVexjXw8yJUkQ-5K1glfPRWeK1fhptPpDu8zezoN6O9sbbx9cq5pfqDc5INwdG72Ok-0gHjf0vg5BPjPMIlGsMBZuNa5Z5ApVMuZxT_vNfOnH2zHDfRy6_FwJif64vfZjIQOPaBwiQ9ZhBB6sHw9prhxMvkMlQ=)  
- [startribune (coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHsZk2aAOb2Z9WgUYACntbwKIHx_5hnzQ87I3dK1AM_w_Z771vQROyw3yxlkd8N1KljcptAS7PynNHFX4LMcjOqxRo0R0N-36W_Ubv0Np6C9wMjBbrfWfnpM7luT1J5XRMg9erDXPYy7Qc54IYZimltjcvx-qTCzy49i73iwQcEftfih9-fWe1ZWvCArYH0RstJP-VxKGuSxpZF6FkNyI8qwj7jcspta6KvGKYnMrMM5oOJojXUcZ4p1ojCTfKnwA==)  
- [stocktitan (SBTi summary)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEP1A2hMC4tnCOH3CBt_BBlx2w1F1weSMSyY2BT6Z78XqSAh4BylVfkBcJWDoGZ6k6iuCD8DfzDGqLND9SwVPvvmJ0N8yuS1WKMCwZXXV8Y71IgQGziGCOcOq3gUxS57h_FpPQ5d4dRO-2MQItqPC2VkLLkyXs_6Afw9-ru_0hoHs1JwIJAkKme9JkTKQOL19feBpXP_nowMmYgK7fetO0NyApQTmFfM34=)  
- [smartenergydecisions (targets)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFqAbvUaohpYUnqUUpnzUGLviuk67s25Rb5J5Q9cV34UWk2pw5eOw-KBvmZ5Udfsv7XP6DSulounqr3IgzPY4IhFdJr1uIBnT-8O7maveEvAIPthnd9tuLx9B_ICTmELEkciRKj3ZTSohuWkBzgpxnXjFSuQAsPkUiYh6b0OrBwFwo=)  
- [opensustainabilityindex](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEVM9_vWveWIAWIWlqhUEt1v7C27Wcwlgm75IAH4JHmywcEs2u0dqBgQ9B5sgVOxUTLhJqb8rQ8y1emcqtPVO90tH4Avnv0dB6VhVoSHUtDRqzvkbQdxJSK-BiCnIvUn4bhO5buUzBdNi4vy4U1)  
- [3m (scope 1 stats)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEWPpT7yhWJR5jdO7OLLdfPk--wm7gzwWOFNseRwCWliJrByLA_fSHofUf7Dplitl-fliYzX1iGyHt2XOGraQUQ1kwFveFdACcu3b3DcdOUzLR9pb_958nlzGEfM_145hD2IcdnDyjAI3l_t19JJ91AbKWgoKKQmdn7CpE7UeeObfzISNKvIwvxlJ5hQX7CnJBF3nYdfPfRQVq7wStXbOvhR470f5Q=)  
- [reset-connect (metrics)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZgT_m_JyDjRjPEUR13HyCJ5gA-mQEhM7GWSWJ_wDFAMb03OqHWVQSzprQ3mHfCqNdNb9__g0zZdC2XicIrSRj8V-MG95zduau8rE5KGhjj6MtGAR7njeJPg0TJSBQin7vCv42JP0=)  
- [esgtoday (scope 3 coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEad8CPrn67YvhijWxnNHV47_nH2L2_llExUcLFgTkBe4_tbfhWhjnrMf6oiHV8H2e90Cw-dul0UD3M_tec1SYXyodZmqRLmBmtqwho_qr29hG2TWjdgT8yALSahWy87pnkuGZV3Xh3lpKZUgCZwPlOU3SS4Utt417Gqjf96YUA3YRD2UNefp8dZbIInwqXtvHxzggxeSpNcrmlAnUbPrKACfrCGYR)  
- [packagingguruji (plastics target)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFP6tY1EllhiXV-K3SEXispIR8EiE6ANZQpWwCBD22fTvnUljMgtTZ_taIO9pXUxlBwbfN29HMg28MQMG9dThyFTIz3nHw-zyWgU3ZafK5S7MDRMow2hz31ieKncKYGMECApyEhVhx-IVnwS51kvRiTK3ScsqN1OOw=)  
- [energyindustryreview (plastics/CBG)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH15VBUdTf7a_YX66JOSIReVyzI4mJv7YrGnO03AD4ybgJ2UwAc5U-q86Shxr8WpIGnNKx7Wl5ERxRVem7asZzVLumNeiUswjuD_x54IV1gT1QF6j31QKRoHJ7HONYVvsFPle6uBfceDiol7ti-ay96cjIFFNkaeKBJjr7YZETdVS-5ADrz1tb9Azmw4eGN_GPLj8wm5iUltXLZJ1vTZxLV8ELJ9UiLd78LW035AI92P92rbg==)  
- [homedepot (case example)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFrPaWFLzPpuh1CQWK3B5e_mtmOQXal9T26F5tYvf1ulhuzLgYLDSHWxVLhggIefsYgwxHkz7xZGebEodBKKNV-CaD_HK10sJ2u119nr7I1S7_3QdVmrxAlSx1_m55UNj8aQcO2SrKF97OVO6TlxJf65jRxZRrdSXZTQ0SB427aTnquNtA=)  
- [supplychaindive (PFAS)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGduLVEw6KHW2v31fzGxjwl6Mj_WRYFVRDCggm1YSVf20FWe11lVjxBtAstObXkR-m4_52Vr0bY1Kp3o4hj75u-Pq-1DBEhB0nln2a9FL_-jqbP5MMaOoT1F_VXIkvCq2Oaus4KYXlwvNX8NUq7fyCmEyT2IMN_JMZr9sQbaDPu7Z2MHIWXK0aP9QEbnfCe2JCa2YucDf0Aw9ijQCLdVr0M4eZT-tMQXpZZxms=)  
- [soci (PFAS)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzksoJ3NqEi3C0Ftyn0oEApPEpQbUrWzMp77TYjcBhEg3a5JYWT-LZlT5Shltu8ssfazSvbn7TSFcV5GLCUITrGGURO1Jv5VdcozF6dmedYpIXOiN7wGAJPcNNh8aA1TiLjaxOyhE_3DLts0ecR8ol1Q8r1mzQgif6LG_GtOuBPOur6L1WkQO2Q0jU3DdZcDk=)  
- [kare11 (PFAS local coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0kiT3frH3eyX3M3eaY1-pyAwGwSiz0paq6ING16SGUWLm4QJ678M_gZ3og21TLJ7YBnxPvT3MT-4ktPR4oOKcxnIWfk-tR7zj4xJ4eOO1HXLNjq4ITAJzXuAkosLkawLcfbUa6aWkmqn9PyJsGBbImyVZnjuUEmWJV6KYY_U8VppBQy0nG1b9DNOYXi0GP2LZeI0AlTBY4efrDggqTPWSGB80YywDUMcQsb1pyT5t5zi7uVsukirDfTPPQ68X)  
- [toxicfreefuture (PFAS context)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEgIFuwnHL_6pFT3-GdNChLW7h1f-FxzIx6wsNKSxmCDb7icqQsoSia2GZAAWU9EnYGTqpYtTHwNn-NJNqUZi98s0NGx0-2cZ9WvCkGsv94Dv2NQCPl16JeZ03ENWeZDjdMciAoSb7-FCMi2512Yvq4ChzvpjlsHdoGBRsltlWa5Kk0uqF2PG71lMd6YyM7j027ULVAfjU5Rn3MIE5mCQUh65c=)  
- [investigatemidwest (PFAS reporting)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEBFNDRyqdAT2Q48Pmo9SVUxepfEC1TSNldLkD2b7i8UPRPBDe5hOHg5ejD5RyZp96udaIs1Rqyv6YriTxpM9hbaM3H_LQ3LtF9BzAKZQ3WBgIGCWSzfZlBVP5I74hqrbQ-x_90-u58DjdYWKKcpiUeyMXEFHHSROApXmVH1Lc1PAAcuCEK-roQDq2FdWIZx8yN70C0cgoi8ukWXz69w-w)  
- [6-0 summary coverage (benchmarking)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEgnFqC5vvVwHDNDn8gFyLTQJkPOAPa1OT6S_nCBXh5qpR7yldb_6kGGbhsd0EREHOc14qSt2GLeBpzArmXj2OUQuotl29vi8lsp-nJm_UsYMyXC6K66P9fzh9cI3uloCdsmHPki34JE-eY-t_R6Q9j_UnPVJsvmxVfKsCovy0OmcTFK2BHqPBVVj9iTQi0ydoaB1LjdcHD2bo8XAqTSLabLEfaiIUrQLzY0x-55gp98TuoDs3TYwh8v1qmNg2wDxShKBwnIQG0l8IrsOH3WpKAbiPKk09XEw==)  
- [esgtoday (peer summaries)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHhGmihWcjcs_6W1FEx_nUPXxn3kpjWY4oDgcumIGRx5dyFlDlHEqzFbgxIRoiBaHOvSlGWBOYtBwzbNa9oZf4Kdr6vTKKmYy5opJak2X01uYdtIQuAthek51ZeLbSAE-F4ECmLjcn_cz8muB2P9hYVL2KZ5v3vBGfMopyOQIq20Si1LRg2ObfSWhMbRmNUdvnVcqDAQZDpBZzsgX9vhyKznZxZIAY6)  
- [smartenergydecisions (peer data)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEARRlo28elVDWakX_z8jU8vTO8tazWy3NLr5OW0hWWzun7CukZqrDJFmlugWVMiTr0-OQzc2hrwxxTboW1nd-xCpoL7LLC3LYhCH5Mn8eQzW-QgUlhRUwwbnhtMAt_6CYyYcIjLuNABBFmHf07Y1T02HJRoVCjCbc1Sjwhb6Pllto=)  
- [startribune (peer coverage)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGN0wA4CTzpTVKEFSmtapWjUt9QvaqNfBPO0Zl6UbxUMffUcGPN3hGHLZNFBS9zhQRu_45x3XpS1x08Y5peIH4PDt6zEGD6enoawNY8J3-UL96fbKeed2-DL6iFbaRJJvkGA8BSx6f1AlU3JZyT_FH0ZMTM16hKAQ6hVTLXlVLikba3SvoFfwEqqcwmGmjr_AqEE0OJL6FqQkLuHS9BDz4=)  
- [environmentenergyleader (Honeywell/peers)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGVoRoo8T8gKXzzEe-GkDVGohPFEq_f_xWkEd24sMMUAOwwKPiExcI5JGuhqUeV4HwLGh3Njh9enpkNKoE8Xy3DZ-vImIrzcY5mFi7aBTHRU7zUTo6KBL3rUDBFZsmp64A5AitGfQk-mFAZwL1a5TkI6iwR13cxCpmipo4S3vpeYK4rulwH3zMAeIYcg4mRrzfuLN_vMU3UgcYuQTG9mYvXBcA=)  
- [honeywell (commitments)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4ilCXc0lgeN1xwdDmGF2a_TM0XK14vgJGoc7HmtoHgi1RCLEB7RjKMlHrhekc96U5Kvo3EQZZVM90hhe7gJUyXFtmhWMEXlQ62-7ahEsl0wuNM5jYeXMXQcOmsAzMC8JpWN6V0512apd8FgAy48_9f_S8XLbw2fVwWQwaZ6AGtcMPMEGO_OsNJcAn8MSYxioIVOzmA6sIq4bOQA45CuMtXpDPuwTQZPlGybXaj4dm_ec1CsLfla3_3ioWtgSl3kd-IiA4_hnsZw_zsy9oJq9rnRalz1Eii0Kcep3w)  
- [dupont (sustainability)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQED-hUtWQC5CWJo_0r2JS_be-Skvy2eBIojAImAvAQDYpfIBKm-QA2y6D3mlyQT8zgyNTdyufwoZH2pp1eWyb3fVnkHLAKMDmzEBUXu-9Z5ys_a93T7q3mYE5qVqPcvJ9wjysVb21RCoDYOebPnXDQs9LAzpEO09Qt5L5OoarcuETIY-QM1tVWP0X0uI4jy70iHROKgyRM6mNSBD5tRReXwDz3DpqRI3CVsxp264xbqQHxrDw==)  
- [specialchem (dupont reporting)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEasODj1SGdbCaNdZsI6-wrMVZuYL5iNScHqFYxIYzSIXT-hGlZtRiUJQFNoF35QCKUxetazcIJ0EijLDTin0iKLU3tJIzHPOfiT7z2wcOMCgQdb6L5z_fe-0aLnOb0db51AnqRD_RajWGGMrQS0VnzXR1ZdwJ23Rn3tuwtZAQQCfLr40L7ORYxRsvQTFdGf252n2viS5iOP2J2hXA=)  
- [ecolab (targets)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEumPC3QuNKwFZjFdo4903futNtmF6eY_hRFvjJ6mg8tpVJAJQ-is41ful-Tl8GUETLJIGLCGNw2ZZVf6faXnPyMGCkPyvQkXbnrYTA8SkgX690mJC9-KIdSSGs00m5vsQRR94Jq5tlLIvAnDBOzBA305I4QOt5lz9ZPYGkc2FpWb5-qou1HEA9RhgDAHzUdRChx1dn9YZ0XE22kOpmLkAtwUUhlg==)  
- [jnj (targets and reporting)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGaLvwzbPisrAHRIyHcXShFNnxumsz9qRHGeqNTSHK9MI1WZznaSpMAPwOz6pcsUMQc_FQ2KxqQfnMU-ewNSk-oaMDU5Eu9Dcwy8wqZcvdzpbLbD3jBGyNCUwM3b98IhpUcylSrgGuYe1-jhWEm8ElzwsvLryBe6IAKcg==)  
- [procurementmag (jnj)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF3AkD3v-iWIIi-4pg_0iVwCczQzQlw6Q7xuoy0oMDcWW-M87W83UM3-znzYhiuQGTCmbPv6NVBWEor6T-vGECWkvBaEVzhqmN42QzOCwNf4BX7oowObWJUAMHB9_0kqo66C0ky-0PHfboXE7A9PUApGDVCCwM254HcW9AMZUtfDxnolQoPwA==)  
- [thesustainableinnovation (peer analysis)](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHwHIiSSb-jZNBfqX2L_p1-bVUt4P-kWTDv-vREr9t1LNUgfn26gZD5TGkP6_8ADT2_2D0tBrYMHiy3gigiDcI8BL0bpsH2UB0Hy-iVgwVkfOGPs9x5LdRNhyYI3I_PTlQiE8YIBsVkCxn04R_ZOCy9)

Notes, caveats, and next steps
- Public disclosures are strong for targets and aggregate outcomes (GHG, water, plastics) but are sometimes limited on technical specifics (e.g., exact MW of newer PPAs beyond examples, on‑site CCUS capacities, or detailed patent portfolios). Where you need deeper technical verification (e.g., exact PPA MWs, locations of water‑treatment installations, or patent numbers for specific climate technologies), I can pull targeted primary documents (SEC filings, specific 3M site press releases, patents databases such as USPTO/WIPO) and compare peer filings for a tighter benchmarking exercise. [sec](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcyWj5gZLA_t08NtjB-HbTyiV44d8-KFEkiY39Y80BLcApQOi2P1uefAgzZliKvK9Qe_y8i3fXncPFN9sfmxxoZeUiGPAeKWU5Fao46-EXru5gGYEuMgQdP1hCRU_81RkpnSR9UPSfrDQYoeNDbHu9ZC1u2BvZ2aEsr_agP6c7CUpcpbcm-UjkjFqWSzZw_5Ky) [wipo](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHn5cs0WkcP_CoEPdxOAxvTJrlGYl1343Z0qZofhYlY1AvEtB0SQSpmE7p3-ouFKPvGO5_czNonmWxxhwM4T_FqSjw0pIRF-IYoupJCbM6BcKD551yy6VkNLdasItq_R1gN)

If you want, I can:
- Prepare a short briefing that extracts the specific pages/figures from 3M’s 2022–2024 Global Impact / responsibility reports and the cited press releases to substantiate each metric (useful if you need source pages for audit or presentation). [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZzwpXgxmOfqeUr734d0bvqVKNprjvKJwkUjj_LGjzJSbmg2hBj-4XLfgDdZAm4Swt8-1p6QVjzCEcahvm_4Ujk6JJor9qmTdCFafcsGreq1KUsURp_rD40yxYBK7NRjrGVbIRq2U2C9wbgvjFxWz4DG34pflALHq-ttFQt_MpcEpkRxmbSVId) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErFZ4u3qdz7NWRrWSpq0jT9AvMRBGuQzLeFyMJ0o5D175HtXqXWbNNIwAoIm0QcIugqrIeQ8v9R339JYsfyZbE_9E5_s-Qu1SNtg5MwgIY2BnR_jotUJ1KUyFDCLJ0f0GMcGFC4grHovtRbOlyhvliLBQwmDCpi3sBi8SO_06HOpoG9ELpzJw3by6Au_Xx5xZaF2888dyEN8kZzA==)
- Run a focused competitor metric table (renewable %; scope 1/2 reductions; scope 3 commitments; major PPAs; notable climate‑tech partnerships) for Honeywell, DuPont, Ecolab, and J&J to benchmark 3M in a one‑page chart. [dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQED-hUtWQC5CWJo_0r2JS_be-Skvy2eBIojAImAvAQDYpfIBKm-QA2y6D3mlyQT8zgyNTdyufwoZH2pp1eWyb3fVnkHLAKMDmzEBUXu-9Z5ys_a93T7q3mYE5qVqPcvJ9wjysVb21RCoDYOebPnXDQs9LAzpEO09Qt5L5OoarcuETIY-QM1tVWP0X0uI4jy70iHROKgyRM6mNSBD5tRReXwDz3DpqRI3CVsxp264xbqQHxrDw==) [ecolab](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEumPC3QuNKwFZjFdo4903futNtmF6eY_hRFvjJ6mg8tpVJAJQ-is41ful-Tl8GUETLJIGLCGNw2ZZVf6faXnPyMGCkPyvQkXbnrYTA8SkgX690mJC9-KIdSSGs00m5vsQRR94Jq5tlLIvAnDBOzBA305I4QOt5lz9ZPYGkc2FpWb5-qou1HEA9RhgDAHzUdRChx1dn9YZ0XE22kOpmLkAtwUUhlg==) [jnj](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGaLvwzbPisrAHRIyHcXShFNnxumsz9qRHGeqNTSHK9MI1WZznaSpMAPwOz6pcsUMQc_FQ2KxqQfnMU-ewNSk-oaMDU5Eu9Dcwy8wqZcvdzpbLbD3jBGyNCUwM3b98IhpUcylSrgGuYe1-jhWEm8ElzwsvLryBe6IAKcg==) [honeywell](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4ilCXc0lgeN1xwdDmGF2a_TM0XK14vgJGoc7HmtoHgi1RCLEB7RjKMlHrhekc96U5Kvo3EQZZVM90hhe7gJUyXFtmhWMEXlQ62-7ahEsl0wuNM5jYeXMXQcOmsAzMC8JpWN6V0512apd8FgAy48_9f_S8XLbw2fVwWQwaZ6AGtcMPMEGO_OsNJcAn8MSYxioIVOzmA6sIq4bOQA45CuMtXpDPuwTQZPlGybXaj4dm_ec1CsLfla3_3ioWtgSl3kd-IiA4_hnsZw_zsy9oJq9rnRalz1Eii0Kcep3w)

Would you like me to produce the supporting source extracts and a competitor comparison table next?

# Test Import Sustainability Agent

Here we test the importing of the code located in the parent directory.

In [13]:
import sys
import os

# Add research_agent to path
sys.path.insert(0, os.path.join(os.getcwd(), 'sustainability_research_agent'))

In [ ]:
from configuration import Configuration
from graph import graph

config = Configuration

state = graph.invoke(
        {"messages": [{"role": "user", "content": formated_prompt}]}
    )
report = state["messages"][-1].content

In [5]:
from IPython.display import Markdown

Markdown(report)

SUSTAINABLE TECHNOLOGY ADOPTION SCORE: [89]/100

Technology Adoption: [23]/25
Innovation Leadership: [21]/25
Measurable Impact: [23]/25
Future Strategy: [14]/15
Industry Position: [8]/10

Overview
3M demonstrates a broad and ambitious set of sustainable-technology initiatives spanning renewable energy, energy efficiency, water and waste management, sustainable materials, and chemicals strategy. The company reports substantial, verifiable progress across Scope 1-2 and meaningful Scope 3 reductions, rapid advancement of renewable energy in operations, and a structured future roadmap anchored in its “Science for Circular, Climate, and Community” focus and the 3M eXcellence operating model. Notable strengths include a validated near-term carbon target by the Science Based Targets initiative (SBTi), a public commitment to carbon neutrality by 2050, aggressive plastic-reduction and water-efficiency goals, and PFAS remediation efforts. Some benchmarking against peers shows 3M performing strongly in renewables and emissions accountability, though direct, apples-to-apples comparisons depend on methodology. Key 2022–2025 examples and metrics are cited below.

Key Technologies Adopted (25 points → 23)
What technologies has 3M implemented at scale?
- Renewable energy and energy management
  - 3M achieved 50% renewable energy in its operations ahead of schedule and surpassed its 2025 goal; by 2023 it was at 56.2% renewable electricity globally, with 2024 data indicating 62.9% of total electricity use from sites powered by 100% renewable electricity. This reflects broad-scale deployment of renewable power across global sites. [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
  - Renewable energy targets include 100% renewable electricity by 2050, with substantial progress toward the 2025 milestone. [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExk5kNwlFlid5OHmfTREiV2jYxpBuwFO19GJMCQnpFv-gq7luNdXSIHL5oJr3MPfzdVzp2v5qt_pX5QhVz8TzsQvCt7XSCmlc5D5g4MmKbXBk-djWWhWFg825RFl9j4znm2T3QLDd7B1zjpzkORZXnSrAOPGLi7A1FOpuJVPhXqYXU3hGeL2ifF56TXYfhMbx9nQe5dMNdIk2h), [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj)
- Water purification and efficiency technologies
  - 3M is installing advanced water purification technologies at its largest water-using locations, with completion/operationalization expected around 2024. Water efficiency improved by 21.4% since 2019 (through 2024 data). [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [startribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)
- PFAS reduction technologies and remediation
  - PFAS (forever chemicals) plan includes stopping PFAS manufacturing and use across products by end of 2025, plus investments in water treatment (Granular Activated Carbon, Reverse Osmosis, Ion Exchange, Thermal Oxidation) to reduce PFAS discharges. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- Plastic reduction and sustainable materials
  - Virgin fossil-based plastic use reduced by 98.2 million pounds since 2021; targeted reduction of 125 million pounds by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- Waste reduction and circular economy
  - By end of 2021, 45.2% of sites had zero-landfill status; a 10% additional manufacturing-waste reduction target by 2025 is in place. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEYNhkbikxGKhxGNj3_MztbUCnnvdFcWRbDmdWVshiv1nt17om57DSE3JrvnwJqNaDGxT4skyscaj9ML33hhUsRJgDw2aWl31aSHOKrhFNVWBmKARctRgom4JQUNPNu917gGsMnb_adqxUAZQueNmANh14j_a8VYETFSanlig7ieNN2wZN1fpBbOoxjkAWQDGo=)
- Product sustainability requirements
  - Every new product must carry a Sustainability Value Commitment to drive reuse, recycling, and reduced resource use. [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFJc1AURpbdLQ8jfnP182JNegdpR2MoTBsWvqlbOO52ZbMs6mLKEuNhqAW8MURbGkiDoGJQjDQv4jGQLl67Bfw0Siq7cKei6mg2OOnusRIjxcUw7LzOGcyEmN3butf1N2b2aEJdYTNQeov4SQxMIcERoUL6ljEKRQMBw9N2bqtUJTUFBgD04tlF)

Why this merits a high technology-adoption score
- Multiple breakthrough technologies are deployed at scale (renewables, advanced water treatment, PFAS remediation, GAC/RO/ion-exchange/treatment technologies, waste reduction and circular design). These are integrated into operations and supply chains with quantifiable targets. [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)

Innovation Leadership (25 points → 21)
What cutting-edge sustainable-technology initiatives, R&D projects, or partnerships is 3M pursuing?
- Science for Circular, Climate, and Community pillars
  - The three-pillar framework guides strategy toward circularity, decarbonization, and community impact; 3M also joined ED Ellen MacArthur Foundation’s Circular Economy 100 (CE100) and maintains supplier and materials traceability initiatives. [3M Vision2045](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyZXiQbyESiFn7GLg8GVBaJTFzxw8hjthuH9iTJAPqknyHDMTEfFecg71fj9ofp0pJAvnSP31t_nunk-RNJZTP6ywaMVnJm_taS6-QLN8S3npT-WA0KYC3WkegGFVJGVlfP_9cqPe-S0OeoqZeYxWLAW4=), [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW), [CSRJournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPND-1nn4B_XzGpT7Q3y_SOeENf40X5452rIgPYx858jxIU2box7dZVoiT6OcC_c4hx6przR7tsVI_XKKxp1lW6Ry-NsqnN1ge3xCuYuEyOM0xmpcC1qpul2BP0NQjfPnNJV8MyajP_Ido2mFexCv8OhsHqLgj7A==)
- 3M eXcellence operating model
  - A comprehensive operating model focused on margin expansion, innovation acceleration (doubling new product launches), and supply-chain resilience, embedded across the enterprise to sustain a high-performance, innovation-driven culture. [AInvest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEAWnkMGJVGW-B3ok728rFpizJnQl2ceDuIBRjnv0YcvOfIiXPZ1rixECfkq5cWvPVIYSH1bnAe8ghtYxV8aQSsEjN6H5e73InQsASju2XZ3531fa8yKrrZsW_AnAHGPgL1A8PHwYgLhDaKVMoWc_OOL3AfX83I9CGtE5USqHj3RoLDOtzc-0FsltcJgq00h0ac3FZDKLspZi7b1oy8uC50T-MJ8aY2i_t6), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)
- R&D intensity and future product pipeline
  - Annual R&D investment around 6% of earnings with plans to invest about $3.5 billion in R&D over 2025–2027, targeting climate tech, industrial automation, next-gen electronics, and sustainable packaging. The aim is to launch 1,000 new products. [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==), [CircularCollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF8un6wDql3WuH_ntQiAIt18jycKwdFU5dSLx4dGb27TrD_DSk4eOs8i7aAQjN_8lBUPRX11v4sE2AleM_dS6SfnoPTAKBNU25FbMEas7ln7CINPSEp1SSweA2dALYiMPyKo6t4E7iGhj4pN8nRWSKH0RkEIw5wYerr_93qr_rKSsxz8Tn1SwV6VE7nlyl2)
- Partnerships and external validation
  - SBTi-validated near-term carbon targets for Scope 1-3; collaborations with The Climate Group, WRI, and WBCSD; RE100 membership for 100% renewable electricity. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=)

Why this merits a high innovation-leadership score
- 3M is actively pursuing and reporting ambitious, externally validated targets (SBTi), aggressive R&D investments, and a broad ecosystem of partnerships and circular/climate initiatives that go beyond compliance. It also emphasizes a structured, enterprise-wide operating model to scale innovation. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=)

Measurable Impact (25 points → 23)
What quantifiable environmental and business impacts have resulted?
- GHG emissions reductions (Scope 1+2 and Scope 3)
  - Scope 1+2 reductions: 59.1% since 2019; 52.6% target for 2030 (SBTi). Scope 3 reductions: 30.7% since 2021 (aligning with SBTi). 2024 data confirms substantial progress toward targets. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFIBf4B_V9hfyvr1dHg7K7Qymj0XCHr1aPgQUwocOBx2l9z7Ecw_o5IQrRO685iWl3WBs9WL1Vzs_JBgUyzNyr60LLIkcO9aZ06dO-hD_2-fJR3XEnlZ-KAN5dcnhK-8Q_URB26gbDd0s0H0u5ajz1poJ-fzdldj2-ug3WSkXGMR-BjY0tTRKUmA6fjHWZt2smb4n2Yw920h8Wz95tAB1QPVxubWsw=)
- Renewable energy progress
  - Renewables share surpassed 50% ahead of the 2025 target; 56.2% in 2023; 62.9% of total electricity use from 100% renewable electricity sites by 2024. [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
- Water efficiency and use
  - Water use reduced by 21.4% since 2019 (2024 data), with a goal of 25% by 2030; 2.5 billion gallons per year in water-use savings anticipated. Advanced water-treatment installations at major locations completed by 2024. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [StarTribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV)
- Plastic reduction
  - Virgin fossil-based plastic use reduced by 98.2 million pounds since 2021; 125 million pounds target by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- Waste and landfill
  - 45.2% of sites at zero landfill status by end of 2021; ongoing goal for >30% zero-landfill sites by 2025. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEYNhkbikxGKhxGNj3_MztbUCnnvdFcWRbDmdWVshiv1nt17om57DSE3JrvnwJqNaDGxT4skyscaj9ML33hhUsRJgDw2aWl31aSHOKrhFNVWBmKARctRgom4JQUNPNu917gGsMnb_adqxUAZQueNmANh14j_a8VYETFSanlig7ieNN2wZN1fpBbOoxjkAWQDGo=)
- PFAS remediation and product safety
  - PFAS exit by 2025, with remediation technologies deployed (GAC, RO, Ion Exchange, Thermal Oxidation) and a $1B+ investment toward environmental goals. PFAS discharges reduced by >85% by end of 2022 vs 2019 baseline. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=), [PR Newswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGBIqlD8f_T4OZdjwcTJsikPtA_D9DEKAMtTZlrhWbGZKnWVmUvmuolfHlSjcKT6YcSCNLv7g4CMXpthtJ3w-xk5BfedlD1EnUxtWgPzZBKTM7okJNzKFLi9Uxu4EVxAhh8Nq_w1dcuS4rUOLkzpddHbLmYHDLcA1VjLzVXp9uvYLYK7DBLEpC7hxbXLfIe5ijiqjoaafSafkZFrCGKXHgl5ikCKdf4GZhbFOdQwTqrZqiEyMVXNUE70Nyelkgnnw7wEaC_0e1BTFPrGbllj8ee8D0=)

Why this merits a high-measurable-impact score
- The portfolio contains multiple large, auditable outcomes (percent reductions in Scope 1+2, significant Scope 3 reductions, renewable-energy share milestones, plastic-reduction milestones, and PFAS remediation investments) that meet the criteria of substantial environmental and business impact. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFIBf4B_V9hfyvr1dHg7K7Qymj0XCHr1aPgQUwocOBx2l9z7Ecw_o5IQrRO685iWl3WBs9WL1Vzs_JBgUyzNyr60LLIkcO9aZ06dO-hD_2-fJR3XEnlZ-KAN5dcnhK-8Q_URB26gbDd0s0H0u5ajz1poJ-fzdldj2-ug3WSkXGMR-BjY0tTRKUmA6fjHWZt2smb4n2Yw920h8Wz95tAB1QPVxubWsw=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)

Future Strategy (15 points → 14)
What are 3M's documented commitments and roadmap for future sustainable-tech adoption?
- Carbon and climate roadmap
  - Committed to carbon neutrality by 2050; interim targets: 50% reduction in Scope 1+2 by 2030 and 80% by 2040; SBTi-validated near-term targets for 1-3; a climate-transition plan to be published by end of 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEwAzTMyNgDWWI8U2-ALYs8xFlylSq6QrGjDSJVEJLuerfSB-ITBmfVmAu0cROuj5yavS3HPxP8ibB_UZdpGOREYl0GzcO7Q9BpfQdlKqKmQMvZUvAUw5bWLxaZscXI24UQbleQDLAT5E2OuQ9EcCw6XI9Mt5BihmzbuDe_NPopwjZQy0QbEo9QdYEPsDSCYUke1a0VAfyP17FkuS6_CfRte31GCA==), [EnvironmentEnergyLeader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG9oeyp2qE3ha7qlpDP7F3W_Pn6vp4jdU-SbkrtiLssxkcpda-LTifHj2kdhE6UU9ZbjBgHvj7oMBlLUisXERKs2nr9WzlMEV34vVyrtO7b2EDlFxiJJ8UlIH2O-Vb5R84qYgl5Dym2ytVlFxJKOWYEZ94Kosjyy92bfuXxlZGe8jl2lGxi2rQ2QRGCRzIwB2sHNrVAuUSuP6ZRDXqvi6m6MGfHdI0nZrRwiw==), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- Water stewardship and plastics roadmap
  - Targets to reduce water use by 25% by 2030; 20% by 2025 (interim); 125 million pounds cut of virgin plastic by 2025; water purification installations completed at major sites by 2024. [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZetnOWmgi0QXkofHAaqJdw1HycRNa2ug7RTysSq_ZLVdNeu1edDrmG9JuZg36OLXOiDTR-Q67v4dkiV1uNGbc_o_ieCZkpfhf1C5c-hyTvNgDUXCO_TGzrh_7jOAh6U5sV6k61FP2jXKsb_e-1A28BrjkIqwb_i3fiErMPgYQh_Ju9Qo_4W5Xtyj_pHwpgKdGKdfU-Q==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)
- Waste, circularity, and product-sustainability commitments
  - Zero-landfill targets by 2025 for a substantial portion of sites; Sustainability Value Commitment for every new product; emphasis on circular design. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Slideshare](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpSenjrquLPPc1GAqiDU7R1ylFR77uNrQcze_OdhLydKUwbFxsEFKZL1InJH_wbCVQIbeNaZWU6yjb25_kCVyr84BdfsBNgxgOa64H5FpROozNcrydGkmX8h6qR4uJCisc_tNz3ibVotDIkaRIFvFGxczspXZH6jDlTTwT7KsnTN3Z6DrIqZL6aFUoQ8mBzTnsqfbLbxCOlAE=)
- 3M eXcellence
  - A formal operating model with explicit targets (e.g., margin expansion, 90% on-time delivery by 2025, increased rate of new product launches) to sustain the sustainability and innovation program. [AInvest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEAWnkMGJVGW-B3ok728rFpizJnQl2ceDuIBRjnv0YcvOfIiXPZ1rixECfkq5cWvPVIYSH1bnAe8ghtYxV8aQSsEjN6H5e73InQsASju2XZ3531fa8yKrrZsW_AnAHGPgL1A8PHwYgLhDaKVMoWc_OOL3AfX83I9CGtE5USqHj3RoLDOtzc-0FsltcJgq00h0ac3FZDKLspZi7b1oy8uC50T-MJ8aY2i_t6), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)

Why this merits a high-future-strategy score
- 3M has a comprehensive, targets-based roadmap with explicit near-term and long-term commitments, substantial investments, and a dedicated operating model to execute the strategy. The impending climate transition plan (by 2025) and strong governance on product sustainability further strengthen its strategy. [Science for Climate](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH7YP6zbI5-OPIgGfoG0vlaC3gGsd64lQtsxEyz7iKShYmMe8RDR3G0L4xkQ90gURrr-Lmj_SCZYy9Bx4bMte1zDnN9IowiSxdXp9UULnrXh1IO-ssVXN6u351W_a_fgmEvqXVE6wyFFlne8kyXKblUJk6yYmDrNgkakH1nfqUaOed_L9TPfd2_D06Ih3PFqRJlUf16), [3M eXcellence](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)

Industry Position (10 points → 8)
How does 3M compare to peers (Dow, DSM, Honeywell, Corning) in sustainable tech leadership?
- Benchmarking context shows 3M with strong renewable-energy progress (ahead of 2025 target; 56.2% in 2023; 62.9% of electricity from 100% renewable sites by 2024) and robust GHG reductions (Scope 1+2 down 59.1% since 2019; Scope 3 down 30.7% since 2021). In comparison, peers show varied progress: Dow with 40% renewable electricity by 2022 (and 2025 goal), 20% freshwater reduction target; Honeywell pursuing carbon neutrality by 2035 with ISO 50001 adoption in early sites; Corning and Owens Corning reporting significant emissions reductions and renewables shares. While each company reports its own metrics, 3M appears to be among the leaders in renewables share, validated targets, and PFAS actions, contributing to a competitive position in the sector. [Dow reporting context: [PackagingDive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHb_K51S1URb74w6x8ScUPxWrtkkWekpcpGfJzy5rq1-pYfdSCdTCD67Fc7u4Plrrz2gWo8Q11JepsEVDBFIttOav9ozGz7gAspct-ZP-Q_u7P0vLeT3zYgpmIs4MgAYcFi572MksDLB8DLISkzUJeW-NJ9pN-_wJDSZJ6fv-or2H1D43lPSaMh8bbRTHLseD6os8BL-bGJSYcSuoL_0mU=)], [Honeywell Responsibility Reports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHFVIAIxDRGenIuuXRkIIhu2Qnad7Q3RoptxgHF7IINkef3XweYguFGI5PXZtjeBXvrUyXevzHOSmsg0X2Oc0l1OQZ71E7K0lwnYyfoRcSLOCcn7T3Ynxz2a_v-dv6XBQ0Flz1on4D1DWhGyw1LU90gfe-qS6btghp6_Gqi9yrc7md6g-8eoUU9pSkIwq9m8IqKrK6d7Nyd4LnU), [Corning 2028 Scope 1+2 targets](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGM3EDgyX_tAi1TRoXWbjaws4xjuefrPqUJy4io832SS73LCWXoVukASCwz4Quf53xHw_vpT4QeLoGO7xL4AeaSOHhNkeoJ3y3GaLDxDZHqdEi6l8oaXLiegKmFgwFDN-zgK3-gqj_sn_yJ7KbPkJy2_JDP-nC9ainP3SIQmDcjT5A0NXkUYDgXsErymVuhAIRzLrA7VTtJyQw=), [Owens Corning data](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHt9Gt-JLIWyq9v6QPgoBc0ZVapOcCUpwqmIclKo2NKKvCxIkQET_PKFt2gTIG9lbiT7b7t36xcVXAefynjgwvYTBuEZP7mbof42bG_OseUp_Pr86lo1pCiby9Syaz15USIWfLyStLLdQbgP5Mv-jSYAjfNP05i6JuK_IqkwqWi2ZmEU2CGZ_Km7j8EbfdNXs3QPh4rYkvGK6vesIMcDL88_km5Wnron8HqyzK4ByAe4gCk789z7P3nPmmFLXX2KP8=)
- Note on comparability: Companies use different baselines, scopes (1/2/3), and reporting cycles; direct numeric comparability should be interpreted with care. [CDP context](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCbWSmBJq3wh3h69R-uh8HOkJV95rBFLx686B5LFrJLqwuMWSOaPZvItsfH4sbCxxqbh5HsxV52gufx7fkwOCqXHliphwj4AYw6Yn4y1uColPimZ5lwEgrGXRFX38=)

Why this merits a high industry-position score
- 3M shows industry-leading momentum in renewable energy adoption, validated science-based targets, and aggressive climate and water goals, placing it among the top performers in sustainability leadership, even as peers also push strong agendas. [Science Based Targets initiative (SBTi) validation: [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)], [RE100 membership: [Sustainable Energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEh_WmbWIZw8_8k37qPoJ7lB0XPzYPJR3hruaR2bf6TUmbdouA3TCbBBmBGNY0nzxRUNU-2aqsQtzDYWqLKO4yDiWG0dB4jY30W2YECiP3TcNa5RP3Fd86aL4V8nqbYjkZ9A20oU25fXFb7qWnKL2jwrOBL_pebzorOCPrYy7JM2DBfPGzPPa59wXpQ49xtDesFm8hty_AuPZwORCemTeLrakRiY-xUbg==)]

Final assessment
3M demonstrates a robust and forward-leaning sustainable-technology program, combining scalable technologies (renewables, water purification, PFAS remediation, waste circularity), strong leadership through external validation (SBTi) and partnerships, measurable environmental impacts (significant GHG reductions, water-efficiency gains, and plastic-reduction milestones), a comprehensive future roadmap (carbon neutrality by 2050, ambitious 2030/2040 interim targets, and a climate-transition plan due by 2025), and a competitive industry position relative to peers. While there are ongoing investigations into facility-by-facility metrics and more granular data, the evidence supports a high overall leadership profile in sustainable technology adoption.

Cited sources and recent data (highlights)
- Renewable energy progress and targets: 56.2% in 2023; 62.9% of total electricity from 100% renewable sites by 2024; 50% by 2025 (surpassed). [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
- GHG reductions and SBTi validation: 59.1% reduction in Scope 1+2 since 2019; 30.7% reduction in Scope 3 since 2021; SBTi validated near-term targets. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [EnvironmentEnergyLeader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtv4eWupqd3PFqgywBcGVhJRBaIuB9Xku1hgZ9Po4ozB1KsHB_NAtLuDOKnGUwhiheSDtuPVWZM5Cjd79Xc5EMrNmWiqS0HRC3ApeI2xpuDaitakMA4ijLqxDL8yFCdCA6Fe3faF9s8anEyY9D5P6ZySKjkBawHtY-Mb1aLucQXOXqP5LZquF81R8ddNM1Q7WcaZSslqyEuoMBvf84bx_wWerAaaQ267Rf), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHjd-x7HYLWH0_wobaNP5KRTpqOEg0JuVqCxOvOSZWtlQy5rImyfQfTT-MD748R0r-mIz92RAyG2UKKbKvACm62GilwvHAt8Gx210e4fDVbgVo3LhLufGSHQgPr9xzH2_5c_ZsVacIVDUR2yuhYS750JfnPVkNeB4oFhaqcas-aw01pFQczYuJ9mnyE3uLkUdowWx0ZTiudrPT7XVKdiWKxPJn_wxA=)
- Water management and efficiency: 21.4% reduction in water use since 2019; 25% water-use reduction by 2030 goal; water-purification investments at major locations. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [StarTribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV)
- Plastic reduction progress: 98.2 million pounds reduction since 2021; 125 million pounds target by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- PFAS management and remediation technologies: Exit PFAS by end of 2025; deployment of GAC, RO, Ion Exchange, Thermal Oxidation; >85% PFAS discharge reduction by end of 2022 vs 2019. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- 3M eXcellence and investment in R&D: $3.5B in R&D planned for 2025–2027; 1,000 new products; 6% earnings invested in R&D annually; focus on climate tech, circular economy, and next-gen electronics. [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==), [CircularCollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF8un6wDql3WuH_ntQiAIt18jycKwdFU5dSLx4dGb27TrD_DSk4eOs8i7aAQjN_8lBUPRX11v4sE2AleM_dS6SfnoPTAKBNU25FbMEas7ln7CINPSEp1SSweA2dALYiMPyKo6t4E7iGhj4pN8nRWSKH0RkEIw5wYerr_93qr_rKSsxz8Tn1SwV6VE7nlyl2), [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW)
- Industry benchmarking context: Dow and others’ goals and progress; 2024 CDP context; peer comparisons showing 3M’s relative leadership in renewables, SBTi validation, and PFAS actions. [PackagingDive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHb_K51S1URb74w6x8ScUPxWrtkkWekpcpGfJzy5rq1-pYfdSCdTCD67Fc7u4Plrrz2gWo8Q11JepsEVDBFIttOav9ozGz7gAspct-ZP-Q_u7P0vLeT3zYgpmIs4MgAYcFi572MksDLB8DLISkzUJeW-NJ9pN-_wJDSZJ6fv-or2H1D43lPSaMh8bbRTHLseD6os8BL-bGJSYcSuoL_0mU=), [CDP](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCbWSmBJq3wh3h69R-uh8HOkJV95rBFLx686B5LFrJLqwuMWSOaPZvItsfH4sbCxxqbh5HsxV52gufx7fkwOCqXHliphwj4AYw6Yn4y1uColPimZ5lwEgrGXRFX38=)

Notes
- All figures above refer to data and targets cited in 2022–2025 sources and 3M public disclosures cited in the provided summaries. For the most granular facility-level data and interim-year breakdowns, consult 3M’s Sustainability Report, Climate Transition Plan (due end of 2025), and Form 10-K filings. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen), [Climate Transition Plan](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)